# V-polyhedral disjunctive cuts plotting worksheet
1. Table 1: Summary statistics for percent gap closed by VPCs --- avg (%) and number of strict wins (best by at least `EPS`), including set of all instances and set of ≥ 10% gap closed instances
2. Table 2: Average percent gap closed by num disj terms
3. Table 3: Summary statistics for time to solve instances with branch-and-bound

We select instances that meet the following criteria:
1. Belong to MIPLIB, NEOS, or COR@L
2. IP optimal value is known
3. ≤ 5000 variables and 5000 constraints (in presolved instance)
4. The partial branch-and-bound tree with 64 leaves does not find an IP optimal solution
5. The disjunctive lower bound is strictly less than the maximum objective value on any leaf node

There are some instances for which we do not have data for all 6 partial tree sizes. We include these instances in most tables, except if we are showing how some statistic changes as the disjunction increases in size.

# Section 0: Set variables, import whatever is needed, and read in data

### Global variables

In [105]:
## Global variables
EPS = 1e-7

## Set up variables containing relevant directories
import os
repos_key = 'REPOS_DIR'
try:
    REPOS_DIR = os.environ[repos_key]
    print("REPOS_DIR set to \"%s\"." % REPOS_DIR)
except KeyError:
    print("*** ERROR: %s not found!" % repos_key)

VPC_DIR = REPOS_DIR + "/vpc/"
RESULTS_DIR = VPC_DIR + "results/saved/"
DATA_DIR = VPC_DIR + "data/"

REPOS_DIR set to "/Users/akazachk/repos".


### Import data processing, plotting, and export packages and functions

In [106]:
## Import data processing, plotting, and export packages and functions
from IPython.display import display

import pandas as pd
pd.set_option("multi_sparse", True)

import numpy as np
import matplotlib.lines as mlines
from matrix2latex import matrix2latex

import matplotlib.pyplot as plt
scale=2
DPI = 200
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':False, 'top':False})
plt.rc('axes', titlesize=12*scale)
plt.rc('axes', labelsize=8*scale)
plt.rc('xtick', labelsize=8*scale)
plt.rc('ytick', labelsize=8*scale)
plt.rc("legend", fontsize=8*scale)
plt.rc("figure", figsize=[6*scale,4*scale])
# plt.rc("figure", figsize=[6,4])
#plt.rc("figure", figsize=[3,2])
plt.rc("savefig", dpi=DPI)

import re

def tex_escape(text):
    """
        :param text: a plain text message
        :return: the message escaped to appear correctly in LaTeX
    """
    conv = {
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '_': r'\_',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\^{}',
        '\\': r'\textbackslash{}',
        '<': r'\textless{}',
        '>': r'\textgreater{}',
        '≥': r'$\ge$'
    }
    regex = re.compile('|'.join(re.escape(str(key)) for key in sorted(conv.keys(), key = lambda item: - len(item))))
    return regex.sub(lambda match: conv[match.group()], text)

### `initialize_df`: common way to process each data frame that we need

In [107]:
## Common way to process each data frame that we need
def initialize_df(filename):
    """
    Create a multilevel index df out of data from file `filename`.
    """
    df = pd.read_csv(filename, sep=',', index_col=False, skiprows=1)
    df.sort_values(by = ['INSTANCE','disj_terms'], inplace=True)
    df.set_index(['INSTANCE','disj_terms'], inplace=True)
    df.replace({"\'-inf\'": -np.inf, "\'inf\'": np.inf}, inplace=True)
    return df

### `df_ipopt`: Retrieve best known IP objective values

In [136]:
## Best known IP objective values
df_ipopt = pd.read_csv(DATA_DIR + "ip_obj.csv")
df_ipopt = df_ipopt.set_index(df_ipopt[df_ipopt.columns[0]])
df_ipopt.rename(columns = {'IP_OBJ' : 'IP OBJ'}, inplace=True) # for consistency with other dfs
display(df_ipopt.head())
display(df_ipopt['IP OBJ']['bm23_presolved'])

,INSTANCE,IP OBJ
INSTANCE,,
22433,22433,21477
23588,23588,8090
10teams,10teams,924
50v-10,50v-10,3311.179984
a1c1s1,a1c1s1,11503.44413


INSTANCE
bm23_presolved    34
bm23_presolved    34
Name: IP OBJ, dtype: object

### `df_preprocess`: Results from preprocessing instances

In [120]:
## Results from preprocessing instances
df_preprocess = pd.read_csv(RESULTS_DIR + "vpc-preprocess.csv", sep=',', index_col=False, skiprows=1)
df_preprocess = df_preprocess.set_index(df_preprocess[df_preprocess.columns[0]])
display(df_preprocess.head())

,INSTANCE,STRATEGY,ORIG LP OBJ,CLEANED LP OBJ,ORIG FIRST GUR NODES,CLEANED FIRST GUR NODES,ORIG BEST GUR NODES,CLEANED BEST GUR NODES,ORIG FIRST GUR TIME,CLEANED FIRST GUR TIME,...,vpc_version,cbc_version,clp_version,gurobi_version,cplex_version,ExitReason,end_time_string,time elapsed,instname,Unnamed: 137
INSTANCE,,,,,,,,,,,,,,,,,,,,,
22433,22433,536,21240.526170,21240.526170,18,34,18,34,0.344,0.281,...,#78d6a45,#0152c5f,#8bd9396,9.11,20.1.0,SUCCESS,Mon Jun 14 00:07:29 2021,0,22433,DONE
23588,23588,536,7649.866134,7649.866134,2951,940,2951,940,3.545,1.022,...,#78d6a45,#0152c5f,#8bd9396,9.11,20.1.0,SUCCESS,Mon Jun 14 00:07:33 2021,4,23588,DONE
10teams,10teams,536,917.000000,917.000000,130,794,130,794,2.621,12.546,...,#78d6a45,#0152c5f,#8bd9396,9.11,20.1.0,SUCCESS,Mon Jun 14 00:07:44 2021,15,10teams,DONE
2club200v15p5scn,2club200v15p5scn,536,-121.222222,-120.076923,94301,104414,94301,104414,7200.001,7200.002,...,#78d6a45,#0152c5f,#8bd9396,9.11,20.1.0,SUCCESS,Mon Jun 14 04:07:30 2021,14401,2club200v15p5scn,DONE
30_70_45_05_100,30_70_45_05_100,536,8.100000,8.100000,1,1,1,1,7.255,5.671,...,#78d6a45,#0152c5f,#8bd9396,9.11,20.1.0,SUCCESS,Mon Jun 14 00:07:54 2021,25,30_70_45_05_100,DONE


### `df_bb`: Results from generating VPCs for various number of disjunctive terms

In [139]:
## Results from generating VPCs for various number of disjunctive terms
df_bb = initialize_df(RESULTS_DIR + "vpc-bb.csv")
display(df_bb.head())

cutlimit  gomory  mode  partial_bb_strategy  \
INSTANCE          disj_terms                                                
10teams_presolved 2                 -1      -1     0                    4   
                  4                 -1      -1     0                    4   
                  8                 -1      -1     0                    4   
                  16                -1      -1     0                    4   
                  32                -1      -1     0                    4   

                              partial_bb_num_strong  preprocess  \
INSTANCE          disj_terms                                      
10teams_presolved 2                               5           0   
                  4                               5           0   
                  8                               5           0   
                  16                              5           0   
                  32                              5           0   

                              prlp_flip_beta  rounds  strengthen  temp  ...  \
INSTANCE          disj_terms                                            ...   
10teams_presolved 2                        0       1           1     0  ...   
                  4                        0       1           1     0  ...   
                  8                        0       1           1     0  ...   
                  16                       0       1           1     0  ...   
                  32                       0       1           1     0  ...   

                              vpc_version  cbc_version  clp_version  \
INSTANCE          disj_terms                                          
10teams_presolved 2              #fc3db01     #0152c5f     #8bd9396   
                  4              #fc3db01     #0152c5f     #8bd9396   
                  8              #fc3db01     #0152c5f     #8bd9396   
                  16             #fc3db01     #0152c5f     #8bd9396   
                  32             #fc3db01     #0152c5f     #8bd9396   

                              gurobi_version  cplex_version  ExitReason  \
INSTANCE          disj_terms                                              
10teams_presolved 2                     9.11         20.1.0     SUCCESS   
                  4                     9.11         20.1.0  FAIL_LIMIT   
                  8                     9.11         20.1.0     SUCCESS   
                  16                    9.11         20.1.0  FAIL_LIMIT   
                  32                    9.11         20.1.0  FAIL_LIMIT   

                                       end_time_string  time elapsed  \
INSTANCE          disj_terms                                           
10teams_presolved 2           Sat Jun 26 14:14:37 2021            27   
                  4           Sat Jun 26 21:40:25 2021            52   
                  8           Sun Jun 27 06:03:06 2021           736   
                  16          Sun Jun 27 13:14:10 2021           264   
                  32          Sun Jun 27 21:37:55 2021           707   

                                       instname  Unnamed: 273  
INSTANCE          disj_terms                                   
10teams_presolved 2           10teams_presolved          DONE  
                  4           10teams_presolved          DONE  
                  8           10teams_presolved          DONE  
                  16          10teams_presolved          DONE  
                  32          10teams_presolved          DONE  

[5 rows x 272 columns]

In [140]:
## Append results from running baseline solver 7 times
#df = df_bb.append(initialize_df(RESULTS_DIR + "vpc-bb0.csv")) # deprecated
df = pd.concat([df_bb, initialize_df(RESULTS_DIR + "vpc-bb0.csv")])
df.sort_values(by = ['INSTANCE','disj_terms'], inplace=True)

col_list = ["BEST DISJ OBJ", "WORST DISJ OBJ"]
for col in col_list:
    df[col] = pd.to_numeric(df[col])

df['NUM DISJ TERMS'] = df.index.get_level_values(1)

# start = 220
# end = start + 15
# print(df.columns[start:end])
# print(df.dtypes[start:end])

display(df.head())

# Get unique instance list
instances = df.index.levels[0]

cutlimit  gomory  mode  partial_bb_strategy  \
INSTANCE          disj_terms                                                
10teams_presolved 0                 -1       0     0                    4   
                  2                 -1      -1     0                    4   
                  4                 -1      -1     0                    4   
                  8                 -1      -1     0                    4   
                  16                -1      -1     0                    4   

                              partial_bb_num_strong  preprocess  \
INSTANCE          disj_terms                                      
10teams_presolved 0                               5           0   
                  2                               5           0   
                  4                               5           0   
                  8                               5           0   
                  16                              5           0   

                              prlp_flip_beta  rounds  strengthen  temp  ...  \
INSTANCE          disj_terms                                            ...   
10teams_presolved 0                        0       1           1     0  ...   
                  2                        0       1           1     0  ...   
                  4                        0       1           1     0  ...   
                  8                        0       1           1     0  ...   
                  16                       0       1           1     0  ...   

                              vpc_version  cbc_version  clp_version  \
INSTANCE          disj_terms                                          
10teams_presolved 0              #fc3db01     #0152c5f     #8bd9396   
                  2              #fc3db01     #0152c5f     #8bd9396   
                  4              #fc3db01     #0152c5f     #8bd9396   
                  8              #fc3db01     #0152c5f     #8bd9396   
                  16             #fc3db01     #0152c5f     #8bd9396   

                              gurobi_version  cplex_version      ExitReason  \
INSTANCE          disj_terms                                                  
10teams_presolved 0                     9.11         20.1.0  NO_DISJUNCTION   
                  2                     9.11         20.1.0         SUCCESS   
                  4                     9.11         20.1.0      FAIL_LIMIT   
                  8                     9.11         20.1.0         SUCCESS   
                  16                    9.11         20.1.0      FAIL_LIMIT   

                                       end_time_string  time elapsed  \
INSTANCE          disj_terms                                           
10teams_presolved 0           Mon Jun 28 17:00:24 2021            56   
                  2           Sat Jun 26 14:14:37 2021            27   
                  4           Sat Jun 26 21:40:25 2021            52   
                  8           Sun Jun 27 06:03:06 2021           736   
                  16          Sun Jun 27 13:14:10 2021           264   

                                       instname  Unnamed: 273  
INSTANCE          disj_terms                                   
10teams_presolved 0           10teams_presolved          DONE  
                  2           10teams_presolved          DONE  
                  4           10teams_presolved          DONE  
                  8           10teams_presolved          DONE  
                  16          10teams_presolved          DONE  

[5 rows x 272 columns]

### DEBUG

In [141]:
# col = "REF+V FIRST_CUT_PASS"
# tmp = df[col]
# display(tmp)

# for col in df.columns:
#     if str(col).endswith("FIRST_CUT_PASS"):
#         print("{}".format(col))

# inst = 'neos22_presolved'
# col = 'NUM DISJ TERMS'
# df.loc[inst][col]

# display(df.loc[('bppc4-08_presolved',2)]['LP OBJ'])
# display(df.loc[('bppc4-08_presolved',2)]['BEST DISJ OBJ'])
# display(df.loc[('bppc4-08_presolved',2)]['WORST DISJ OBJ'])
# display(df['BEST DISJ OBJ'])

# Section 1: Gap closed tables

### `gap_df`: Calculate gap closed for GMICs, Gurobi, and VPCs

In [142]:
## Calculate gap closed for GMICs, Gurobi, and VPCs

def calc_gap_closed(gap_df, col):
    return np.where(
        gap_df[col] > EPS, # condition
        100. * (gap_df[col] - gap_df["LP OBJ"]) / (gap_df["IP OBJ"] - gap_df["LP OBJ"]), # if condition is true
        0.0 # if condition is false
    )

def calc_gap_closed2(gap_df, col):
    conditions = gap_df[col] > EPS & np.isfinite(gap_df[col])
    choices = 100. * (gap_df[col] - gap_df["LP OBJ"]) / (gap_df["IP OBJ"] - gap_df["LP OBJ"])
    return np.select(conditions, choices, default=0.0)
    
# Create subset of dataframe relevant to gap closed
gap_df = df.loc[:, 
                [
                    'NUM DISJ TERMS',
                    'ROWS',
                    'COLS',
                    'LP OBJ',
                    'BEST DISJ OBJ',
                    'WORST DISJ OBJ',
                    'IP OBJ',
                    'GMIC OBJ',
                    'VPC OBJ',
                    'VPC+GMIC OBJ',
                    'FIRST REF FIRST_CUT_PASS',
                    'FIRST REF+V FIRST_CUT_PASS',
                    'FIRST REF LAST_CUT_PASS',
                    'FIRST REF+V LAST_CUT_PASS',
                    'NUM GMIC',
                    'NUM VPC',
                    'NUM OBJ',
                    'ExitReason']
               ]

# Calculate some missing % gap closed columns
# gap closed = 100 * (post_cut_opt_val - lp_opt_val) / (ip_opt_val - lp_opt_val)
cut_type = "GMIC"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

cut_type = "BEST DISJ"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

cut_type = "VPC"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

cut_type = "VPC+GMIC"
col = cut_type + " OBJ"
gap_df[cut_type + " % GAP CLOSED"] = calc_gap_closed(gap_df, col)

col = "REF FIRST_CUT_PASS"
gap_df[col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)
col = "REF+V FIRST_CUT_PASS"
gap_df[col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)
col = "REF LAST_CUT_PASS"
gap_df[col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)
col = "REF+V LAST_CUT_PASS"
gap_df[col + " % GAP CLOSED"] = calc_gap_closed(gap_df, "FIRST " + col)

display(gap_df.loc['bm23_presolved'])
display(gap_df.loc[("bm23_presolved",2)])

,NUM DISJ TERMS,ROWS,COLS,LP OBJ,BEST DISJ OBJ,WORST DISJ OBJ,IP OBJ,GMIC OBJ,VPC OBJ,VPC+GMIC OBJ,...,NUM OBJ,ExitReason,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,,,,,,,,,,,
0,0,20,27,20.570922,-inf,-inf,34.0,NaN,NaN,NaN,...,0,NO_DISJUNCTION,0.000000,0.000000,0.000000,0.000000,20.867452,0.000000,37.49489,0.000000
2,2,20,27,20.570922,21.483725,27.267238,34.0,22.828302,21.483725,22.960493,...,7,CUT_LIMIT,16.809643,6.797215,6.797215,17.794009,0.000000,20.977280,0.00000,34.778873
4,4,20,27,20.570922,22.537880,29.935573,34.0,22.828302,22.537880,23.250549,...,6,CUT_LIMIT,16.809643,14.647006,14.647006,19.953919,0.000000,21.850177,0.00000,39.502918
8,8,20,27,20.570922,22.936206,35.613171,34.0,22.828302,22.936206,23.250033,...,6,CUT_LIMIT,16.809643,17.613150,17.613150,19.950078,0.000000,21.960083,0.00000,43.457377
16,16,20,27,20.570922,25.881188,35.613171,34.0,22.828302,25.708479,25.708479,...,6,CUT_LIMIT,16.809643,39.543044,38.256964,38.256964,0.000000,38.980155,0.00000,47.717947
32,32,20,27,20.570922,28.161870,44.566830,34.0,22.828302,28.161870,28.161870,...,6,CUT_LIMIT,16.809643,56.526208,56.526208,56.526208,0.000000,57.046658,0.00000,61.994121
64,64,20,27,20.570922,30.168224,56.839578,34.0,22.828302,29.920824,29.920824,...,6,CUT_LIMIT,16.809643,71.466577,69.624303,69.624303,0.000000,66.200094,0.00000,71.325798


NUM DISJ TERMS                               2
ROWS                                        20
COLS                                        27
LP OBJ                               20.570922
BEST DISJ OBJ                        21.483725
WORST DISJ OBJ                       27.267238
IP OBJ                                    34.0
GMIC OBJ                             22.828302
VPC OBJ                              21.483725
VPC+GMIC OBJ                         22.960493
FIRST REF FIRST_CUT_PASS                   0.0
FIRST REF+V FIRST_CUT_PASS           23.387977
FIRST REF LAST_CUT_PASS                    0.0
FIRST REF+V LAST_CUT_PASS            25.241404
NUM GMIC                                     6
NUM VPC                                      6
NUM OBJ                                      7
ExitReason                           CUT_LIMIT
GMIC % GAP CLOSED                    16.809643
BEST DISJ % GAP CLOSED                6.797215
VPC % GAP CLOSED                      6.797215
VPC+GMIC % GA

### `selected_instances_dict`: Select instances for gap closed calculations

Criteria to filter gap closed instances:
* ip opt val is known
* max(nrows, ncols) ≤ 5K
* optimal solution should not be found by any of the partial trees
* either lp opt < disj lb or disj lb < disj ub
* PRLP is feasible and solves within timelimit for at least one of the attempts

In [150]:
## Select instances for gap closed calculations
#
# Criteria to filter gap closed instances:
# * ip opt val is known
# * max(nrows, ncols) ≤ 5K
# * optimal solution should not be found by any of the partial trees
# * lp opt < ip opt and either lp opt < disj lb or disj lb < disj ub
# * PRLP is feasible and solves within timelimit for at least one of the attempts

# Constants
MAX_ROWS = 5000
MAX_COLS = MAX_ROWS

# Information to save
selected_instances_dict = {} # dictionary of (original index, instance)
#selected_indices = []
num_attempts = np.zeros(len(instances), dtype=int)
num_errors = 0

for i, inst in enumerate(instances):
    skip_instance = False
    curr_df = df.loc[inst]
    
    # Count number of times instance appears
    num_attempts[i] = len(curr_df)

    if num_attempts[i] < 7:
        print("*** ERROR: Instance {:d} -- {}: {:d} < 7 attempts.".format(i, instances[i], num_attempts[i]))
        skip_instance = True
        num_errors += 1
        
    # Check that ExitReason != OPTIMAL_SOLUTION_FOUND
    for curr_index, row in curr_df.iterrows():
        #print(i,j, curr_df['ExitReason'])
        exitreason = row['ExitReason']
        if exitreason == 'OPTIMAL_SOLUTION_FOUND':
            print("Skipping instance {:d} -- {}: optimal IP solution found at depth {:.0f}.".format(
                i, instances[i], curr_index
            ))
            skip_instance = True
            break

    # Check that best and worst bound on leaf nodes is not same (likely cause of primal infeasible PRLP)
    num_successful_attempts = 0
    has_zero = False
    terms = curr_df.index
    for curr_index in terms:
        if curr_df['NUM DISJ TERMS'][curr_index] == 0:
            has_zero = True
            continue
            
        lp_obj = curr_df['LP OBJ'][curr_index]
        ip_obj = curr_df['IP OBJ'][curr_index]
        best_disj_obj = curr_df['BEST DISJ OBJ'][curr_index]
        worst_disj_obj = curr_df['WORST DISJ OBJ'][curr_index]
        num_frac = curr_df['NUM FRAC'][curr_index]
        num_obj_tried = curr_df['NUM OBJ'][curr_index]
        exitreason = curr_df['ExitReason'][curr_index]

        LP_OPT_IS_CUT = (num_frac > 0) and abs(ip_obj - lp_obj) >= 1e-7 and abs(lp_obj - worst_disj_obj) >= 1e-7
        DLB_NE_DUB = (num_frac > 0) and abs(best_disj_obj - worst_disj_obj) >= 1e-7
        if LP_OPT_IS_CUT or DLB_NE_DUB:
            if (num_obj_tried == 0) and (exitreason not in ['PRLP_TIME_LIMIT','PRLP_INFEASIBLE','OPTIMAL_SOLUTION_FOUND']):
                # We should be trying objectives at this point, unless the initial PRLP timed out or was infeasible or an optimal solution was found
                raise ValueError(
                    "*** ERROR: Instance {:d} -- {}: at depth {:d}, num obj tried = 0 but lp opj {:.10f} < best_disj_obj {:.10f} < worst_disj_obj {:.10f} with exit reason {}".format(
                        i, instances[i], curr_index, lp_obj, best_disj_obj, worst_disj_obj, curr_df['ExitReason'][curr_index]
                    )
                )
            num_successful_attempts += (num_obj_tried > 0)
        else:
            # check that num obj tried is 0
            if (num_obj_tried > 0):
                raise ValueError(
                    "*** ERROR: Instance {:d} -- {}: at depth {:d}, num obj tried = {:d} > 0 but best_disj_obj {:f} = worst_disj_obj {:f}".format(
                        i, instances[i], curr_index, num_obj_tried, best_disj_obj, worst_disj_obj
                    )
                )

    if not has_zero:
        raise ValueError(
            "*** ERROR: Instance {:d} -- {}: has no bb0 entry.".format(
                i, instances[i], curr_index
            )
        )        
    
    if num_successful_attempts == 0:
        print("Skipping instance {:d} -- {}: best and worst bound on leaf nodes coincide for all trees, or no objectives ever tried.".format(i, instances[i], num_attempts[i]))
        skip_instance = True
    else:        
        # Ensure IP objective value is known
        ip_obj = curr_df['IP OBJ'][curr_df.index[0]]
        if not isinstance(ip_obj,float):
            print(
                "Skipping instance {:d} -- {}: IP objective value ({}) is not detected to be a float value.".format(
                i, instances[i], ip_obj))
            skip_instance = True
            
        # Ensure nrows and ncols is not too many
        nrows = curr_df['ROWS'][curr_df.index[0]]
        ncols = curr_df['COLS'][curr_df.index[0]]
        if (nrows > MAX_ROWS) or (ncols > MAX_COLS):
            print("Skipping instance {:d} -- {}: nrows = {:d} > {:d} or ncols = {:d} > {:d}.".format(
                    i, instances[i], nrows, ncols, MAX_ROWS, MAX_COLS))
            skip_instance = True
    
    if not skip_instance:
        #selected_instances_dict[len(selected_instances_dict)] = inst
        selected_instances_dict[inst] = i

num_selected_instances = len(selected_instances_dict)
print("Total number of errors: {}".format(num_errors))
print("Total number of selected instances: {}".format(num_selected_instances))

Skipping instance 1 -- 22433_presolved: optimal IP solution found at depth 8.
Skipping instance 9 -- air01_presolved: optimal IP solution found at depth 2.
Skipping instance 9 -- air01_presolved: best and worst bound on leaf nodes coincide for all trees, or no objectives ever tried.
Skipping instance 11 -- app1-1_presolved: optimal IP solution found at depth 32.
Skipping instance 15 -- b-ball_presolved: best and worst bound on leaf nodes coincide for all trees, or no objectives ever tried.
Skipping instance 35 -- bnatt400_presolved: best and worst bound on leaf nodes coincide for all trees, or no objectives ever tried.
Skipping instance 37 -- bppc8-02_presolved: best and worst bound on leaf nodes coincide for all trees, or no objectives ever tried.
Skipping instance 41 -- chromaticindex32-8_presolved: best and worst bound on leaf nodes coincide for all trees, or no objectives ever tried.
Skipping instance 47 -- csched008_presolved: best and worst bound on leaf nodes coincide for all tr

### `selected_gap_df`: Gap closed for selected instances, adding 0-row that has best for `V+` cols

In [151]:
## `selected_gap_df`: Gap closed for selected instances, adding 0-row that has best for `V+` cols
## Show the instances that have been selected (and their original index)
## and then set the selected_gap_df as the selected instances from gap_df
## We also set the '0' row to contain the best result for each method
## (including the option of not using VPCs at all)
## and we replace any runs with no VPCs with the values obtained without them

#display(selected_instances_dict)

# Note that the new df has something weird with the index (if you check selected_gap_df.index.levels[0])
# Namely, it has values from gap_df that are not selected instances
# This issue does not arise when reading the index with selected_gap_df.index.get_level_values(0).unique()
selected_gap_df = gap_df.loc[selected_instances_dict.keys()]

#display(selected_gap_df.index.difference(gap_df.index))
#selected_gap_df.drop(['22433_presolved'])

# # Check what the selected_gap_df contains for bm23
# inst = "bm23_presolved"
# display(selected_gap_df.loc[inst])

#inst = "10teams_presolved"
# inst = '22433_presolved'
# curr_df = selected_gap_df.loc[inst]
# display(curr_df)
# # for i in curr_df.index:
# #     display(curr_df.loc[i])

#display(selected_gap_df.index.get_level_values(0).unique())

col_gmic = 'GMIC % GAP CLOSED'
col_best_disj = 'BEST DISJ % GAP CLOSED'
col_vpc = 'VPC % GAP CLOSED'
col_vpc_gmic = 'VPC+GMIC % GAP CLOSED'
col_first_ref = 'REF FIRST_CUT_PASS % GAP CLOSED'
col_first_ref_v = 'REF+V FIRST_CUT_PASS % GAP CLOSED'
col_last_ref = 'REF LAST_CUT_PASS % GAP CLOSED'
col_last_ref_v = 'REF+V LAST_CUT_PASS % GAP CLOSED'
col_num_vpcs = 'NUM VPC'
gap_cols = [
    col_gmic,
    col_best_disj,
    col_vpc,
    col_vpc_gmic,
    col_first_ref,
    col_first_ref_v,
    col_last_ref,
    col_last_ref_v,
]

# Do we update the value of the "best" in each column when no VPCs are generated for a run and we use the "no-VPCs" data?
# This may cause the stats in the "best" row to improve
# For example, we replace V+GurF with GurF when no VPCs are generated, since that is what would occur without VPCs
# But if GurF is better than any V+GurF when VPCs are produced, then the average in the max-row is inflated
SHOULD_UPDATE_MAX_WHEN_NO_VPCS = True

inst_set = selected_gap_df.index.get_level_values(0).unique()
num_inst = len(inst_set)
for curr_inst_ind, inst in enumerate(inst_set):
    print("{}/{}".format(curr_inst_ind+1,num_inst), end='\r', flush=True)
    curr_df = selected_gap_df.loc[inst].copy() # copy needed to not throw SettingWithCopyWarning

    # Set 0-row to have max values across all rows for this instance
    max_vals = curr_df[gap_cols].max()
    selected_gap_df.loc[(inst,0),gap_cols] = max_vals

    for ind in curr_df.index:
        if ind == 0:
            continue

        # Propogate GurF and GurL down
        sel_gap = [col_first_ref, col_last_ref]
        selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,sel_gap]

        # If no VPCs produced, the values for V+GurF and V+GurL have not been provided
        # We replace these by GurF and GurL
        # Currently disabled: update max for that column too (if disabled, we instead keep max as the value among those that generated VPCs)
        num_vpc = curr_df.loc[ind,col_num_vpcs]
        if num_vpc == 0:
            # print("Zero cuts for inst {} at depth {:d}".format(inst, ind))
            ref_gap = [col_first_ref, col_last_ref] # this is where we pull info from
            refinds = [gap_cols.index(colname) for colname in ref_gap] 
            sel_gap = [col_first_ref_v, col_last_ref_v] # this is where we put the info
            selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,ref_gap].to_numpy()

            if SHOULD_UPDATE_MAX_WHEN_NO_VPCS:
                for i in refinds:
                    if curr_df.loc[0,gap_cols[i]] > selected_gap_df.loc[(inst,0),gap_cols[i+1]]:
                        # if curr_df.loc[0,gap_cols[i]] > 0:
                            # print("DEBUG: Updating {} for inst {} from {:f} to {:f}".format(
                            #     gap_cols[i+1], 
                            #     inst, 
                            #     selected_gap_df.loc[(inst,0),gap_cols[i+1]], 
                            #     curr_df.loc[0,gap_cols[i]]))
                        selected_gap_df.loc[(inst,0),gap_cols[i+1]] = curr_df.loc[0,gap_cols[i]]

display(selected_gap_df.head(21).loc[:,[col_num_vpcs]+gap_cols])

NUM VPC  GMIC % GAP CLOSED  \
INSTANCE          disj_terms                               
10teams_presolved 0                 0         100.000000   
                  2                74         100.000000   
                  4                 4         100.000000   
                  8                74         100.000000   
                  16                1         100.000000   
                  32                1         100.000000   
                  64                0         100.000000   
23588_presolved   0                 0           5.772830   
                  2                34           5.772830   
                  4                75           5.772830   
                  8                75           5.772830   
                  16               75           5.772830   
                  32               75           5.772830   
                  64               75           5.772830   
30n20b8_presolved 0                 0          11.513514   
                  2               190          11.513514   
                  4               190          11.513514   
                  8                 0          11.513514   
                  16                0          11.513514   
                  32                0          11.513514   
                  64                0          11.513514   

                              BEST DISJ % GAP CLOSED  VPC % GAP CLOSED  \
INSTANCE          disj_terms                                             
10teams_presolved 0                         0.000000          0.000000   
                  2                         0.000000          0.000000   
                  4                         0.000000          0.000000   
                  8                         0.000000          0.000000   
                  16                        0.000000          0.000000   
                  32                        0.000000          0.000000   
                  64                        0.000000          0.000000   
23588_presolved   0                        72.182376         71.597382   
                  2                        21.886890         18.918235   
                  4                        34.091089         27.647967   
                  8                        43.576348         34.750650   
                  16                       49.529133         48.216085   
                  32                       62.098341         60.599198   
                  64                       72.182376         71.597382   
30n20b8_presolved 0                         1.223891          0.017716   
                  2                         0.010731          0.010582   
                  4                         0.022891          0.017716   
                  8                         0.132031          0.000000   
                  16                        0.185150          0.000000   
                  32                        0.535332          0.000000   
                  64                        1.223891          0.000000   

                              VPC+GMIC % GAP CLOSED  \
INSTANCE          disj_terms                          
10teams_presolved 0                      100.000000   
                  2                      100.000000   
                  4                      100.000000   
                  8                      100.000000   
                  16                     100.000000   
                  32                     100.000000   
                  64                     100.000000   
23588_presolved   0                       71.599240   
                  2                       20.044423   
                  4                       27.647967   
                  8                       34.750650   
                  16                      48.241773   
                  32                      60.621962   
                  64                      71.599240   
30n20b8_presolved 0                       11.513514   
           

#### DEBUG: Why REF+V is less than REF

In [ ]:
### DEBUG
# Why REF+V < REF

inst = 'f2gap801600_presolved'

display(gap_df.loc[inst,['NUM VPC']+['FIRST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

display(selected_gap_df.loc[inst,['NUM VPC']+['FIRST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

In [ ]:
inst = 'f2gap801600_presolved'

display(gap_df.loc[inst,['NUM VPC']+['FIRST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

display(selected_gap_df.loc[inst,['NUM VPC']+['FIRST REF FIRST_CUT_PASS']+['FIRST REF+V FIRST_CUT_PASS']+gap_cols])

### `best df`: For each instance, what the best gap closed is (and how that was obtained)

In [154]:
## Create best df = for each instance, what the best gap closed is (and how that was obtained)
cols = [
    'G',
    'DB',
    'V',
    'V+G',
    'GurF',
    'V+GurF',
    'GurL',
    'V+GurL',
]

map_short_to_cols = {
    'G'      : 'GMIC % GAP CLOSED',
    'DB'     : 'BEST DISJ % GAP CLOSED',
    'V'      : 'VPC % GAP CLOSED',
    'V+G'    : 'VPC+GMIC % GAP CLOSED',
    'GurF'   : 'REF FIRST_CUT_PASS % GAP CLOSED',
    'V+GurF' : 'REF+V FIRST_CUT_PASS % GAP CLOSED',
    'GurL'   : 'REF LAST_CUT_PASS % GAP CLOSED',
    'V+GurL' : 'REF+V LAST_CUT_PASS % GAP CLOSED',
}

map_cols_to_short = {v: k for k, v in map_short_to_cols.items()}

inst_set = selected_instances_dict.keys()
# inst_set = ['neos22_presolved']

best_df = pd.DataFrame(
    columns = cols+[
        'BEST VPC DISJ',
        'BEST GMIC+VPC DISJ',
        'BEST V+GurF DISJ',
        'BEST V+GurL DISJ',
    ],
    index = inst_set,
    dtype = float,
)

num_inst = len(inst_set)
for i, inst in enumerate(inst_set):
    print("{}/{}".format(i+1,num_inst), end='\r', flush=True)
    # print("Processing instance {:d} with name {}.".format(i, inst))
    best_vpc = -1.
    best_vpc_disj = -1
    best_vpcgmic = -1.
    best_vpcgmic_disj = -1
    best_VGurF = -1.
    best_VGurF_disj = -1
    best_VGurL = -1.
    best_VGurL_disj = -1
    
    curr_df = selected_gap_df.loc[inst]
    
    # Get info for GurF and GurL from the no-VPC row
    row = curr_df.loc[0]
    GurF_gap = float(row['REF FIRST_CUT_PASS % GAP CLOSED'])
    GurL_gap = float(row['REF LAST_CUT_PASS % GAP CLOSED'])
    gmic_gap = float(row['GMIC % GAP CLOSED'])
    disj_gap = float(row['BEST DISJ % GAP CLOSED'])

    for index, row in curr_df.iterrows():
        num_disj_terms = int(row['NUM DISJ TERMS'])
        if num_disj_terms <= 0:
            continue
            
        # print("Index {:d}: Processing instance {} with {:d} disj terms.".format(index, inst, num_disj_terms))
        vpc_gap     = float(row['VPC % GAP CLOSED'])
        vpcgmic_gap = float(row['VPC+GMIC % GAP CLOSED'])
        VGurF_gap   = float(row['REF+V FIRST_CUT_PASS % GAP CLOSED'])
        VGurL_gap   = float(row['REF+V LAST_CUT_PASS % GAP CLOSED'])
        
        if best_vpc < vpc_gap:
            best_vpc = vpc_gap
            best_vpc_disj = index
        if best_vpcgmic < vpcgmic_gap:
            best_vpcgmic = vpcgmic_gap
            best_vpcgmic_disj = index
        if best_VGurF < VGurF_gap:
            best_VGurF = VGurF_gap
            best_VGurF_disj = index
        if best_VGurL < VGurL_gap:
            best_VGurL = VGurL_gap
            best_VGurL_disj = index

    best_df.iloc[i] = [
        gmic_gap if gmic_gap >= EPS else 0.,
        disj_gap if disj_gap >= EPS else 0.,
        best_vpc if best_vpc >= EPS else 0.,
        best_vpcgmic if best_vpcgmic >= EPS else 0.,
        GurF_gap if GurF_gap >= EPS else 0.,
        best_VGurF if best_VGurF >= EPS else 0.,
        GurL_gap if GurL_gap >= EPS else 0.,
        best_VGurL if best_VGurL >= EPS else 0.,
        best_vpc_disj,
        best_vpcgmic_disj,
        best_VGurF_disj,
        best_VGurL_disj,
    ]

display(best_df)

,G,DB,V,V+G,GurF,V+GurF,GurL,V+GurL,BEST VPC DISJ,BEST GMIC+VPC DISJ,BEST V+GurF DISJ,BEST V+GurL DISJ
10teams_presolved,100.000000,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,2.0,2.0,2.0,2.0
23588_presolved,5.772830,72.182376,71.597382,71.599240,14.222797,70.957296,26.386033,71.826753,64.0,64.0,64.0,64.0
30n20b8_presolved,11.513514,1.223891,0.017716,11.513514,1.234311,1.312801,17.285869,28.956262,4.0,2.0,2.0,4.0
50v-10_presolved,45.753596,18.008191,6.836095,45.823184,50.218750,50.861824,70.906623,74.653591,64.0,16.0,2.0,4.0
a1c1s1_presolved,25.100614,4.895611,1.820497,25.386388,45.998106,47.072835,88.344774,88.650009,64.0,8.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
umts_presolved,0.973181,0.209040,0.109528,0.973181,1.302534,1.368979,4.731895,5.672506,32.0,2.0,2.0,16.0
usAbbrv-8-25_70_presolved,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,2.0,2.0,2.0
vpm1_presolved,29.906542,7.242991,7.009346,29.906542,41.121495,50.934579,50.934579,50.934579,64.0,2.0,2.0,2.0
vpm2_presolved,17.849671,14.293216,8.125084,20.006451,42.919339,50.063401,75.669712,71.193532,32.0,32.0,2.0,2.0


#### DEBUG: In `best_df`, can get V > V+G due to numerical issues

In [ ]:
## DEBUG: You can get V > V+G due to numerical issues

col1 = best_df['V']
col2 = best_df['V+G']

display(best_df[(col1 > col2 + EPS) == True])

df.loc['neos-1058477_presolved'] #.to_csv("neos-1058477_presolved_data.csv")

#### DEBUG: Find instances in which V+GurF max does not match up

In [161]:
## DEBUG: Find instances in which V+GurF max does not match up
# This causes the value in Table 2 'Best' row to not match Table 1 'All'

# For instance f2gap801600_presolved, the gap closed at the end of the root node is 0% whenever VPCs are used,
# but without VPCs, the gap closed is 50%
# In `best_df`, for an instance in which no VPCs were generated,
# we use the value of GurF/GurL for V+GurF/V+GurL
# In `selected_gap_df`, the "zero" row contains

col = 'V+GurF'
origcol = map_short_to_cols[col]
num_errors = 0
for inst in best_df.index:
    val1 = best_df.loc[inst,col]
    val2 = selected_gap_df.loc[(inst,0),origcol]
    if abs(val1-val2) > EPS:
        print("{} has best_df = {:f} and selected_gap_df = {:f} for col {}".format(inst,val1,val2,col))
        num_errors += 1

print("Total # of errors =", num_errors, flush=True)

Total # of errors = 0


In [162]:
# ## DEBUG
# gap_cols = [
#     'GMIC % GAP CLOSED',
#     'BEST DISJ % GAP CLOSED',
#     'VPC % GAP CLOSED',
#     'VPC+GMIC % GAP CLOSED',
#     'REF FIRST_CUT_PASS % GAP CLOSED',
#     'REF+V FIRST_CUT_PASS % GAP CLOSED',
#     'REF LAST_CUT_PASS % GAP CLOSED',
#     'REF+V LAST_CUT_PASS % GAP CLOSED',
# ]
# col_num_vpcs = 'NUM VPC'

# inst = 'f2gap801600_presolved'
# tmp_selected_gap_df = gap_df.loc[selected_instances_dict.keys()]
# curr_df = tmp_selected_gap_df.loc[inst].copy() # copy needed to not throw SettingWithCopyWarning

# # Set 0-row to have max values across all rows for this instance
# max_vals = curr_df[gap_cols].max()
# # selected_gap_df.loc[(inst,0),gap_cols] = max_vals

# display(tmp_selected_gap_df.loc[inst])
# display(max_vals)

,NUM DISJ TERMS,ROWS,COLS,LP OBJ,BEST DISJ OBJ,WORST DISJ OBJ,IP OBJ,GMIC OBJ,VPC OBJ,VPC+GMIC OBJ,...,NUM OBJ,ExitReason,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,,,,,,,,,,,,,
0,0,80,1600,86570.120867,-inf,-inf,86679.0,NaN,NaN,NaN,...,0,NO_DISJUNCTION,0.000000,0.000000,0.000000,0.000000,49.794634,0.0,49.794634,0.0
2,2,80,1600,86570.120870,86571.27164,86578.45463,86679.0,86655.69102,86571.03894,86655.69102,...,75,CUT_LIMIT,78.591875,1.056924,0.843201,78.591875,0.000000,0.0,0.000000,0.0
4,4,80,1600,86570.120870,86573.01780,86582.74713,86679.0,86655.69102,86571.18806,86655.69102,...,34,SUCCESS,78.591875,2.660684,0.980160,78.591875,0.000000,0.0,0.000000,0.0
8,8,80,1600,86570.120870,86574.09838,86619.55332,86679.0,86655.69102,86571.32682,86655.70780,...,6,SUCCESS,78.591875,3.653143,1.107604,78.607287,0.000000,0.0,0.000000,0.0
16,16,80,1600,86570.120870,86575.01890,86619.55332,86679.0,86655.69102,NaN,86655.69102,...,20,FAIL_LIMIT,78.591875,4.498594,0.000000,78.591875,0.000000,0.0,0.000000,0.0
32,32,80,1600,86570.120870,86575.75367,86619.55332,86679.0,86655.69102,86571.27164,86655.69102,...,87,CUT_LIMIT,78.591875,5.173443,1.056924,78.591875,0.000000,0.0,0.000000,0.0
64,64,80,1600,86570.120870,86576.73819,86622.23104,86679.0,86655.69102,NaN,86655.69102,...,20,FAIL_LIMIT,78.591875,6.077675,0.000000,78.591875,0.000000,0.0,0.000000,0.0


GMIC % GAP CLOSED                    78.591875
BEST DISJ % GAP CLOSED                6.077675
VPC % GAP CLOSED                      1.107604
VPC+GMIC % GAP CLOSED                78.607287
REF FIRST_CUT_PASS % GAP CLOSED      49.794634
REF+V FIRST_CUT_PASS % GAP CLOSED     0.000000
REF LAST_CUT_PASS % GAP CLOSED       49.794634
REF+V LAST_CUT_PASS % GAP CLOSED      0.000000
dtype: float64

In [163]:
# ## DEBUG
# #inst = 'f2gap801600_presolved'
# inst = 'neos22_presolved'
# display(selected_gap_df.loc[inst,[col_num_vpcs]+gap_cols])
# display(best_df.loc[inst])

,NUM VPC,GMIC % GAP CLOSED,BEST DISJ % GAP CLOSED,VPC % GAP CLOSED,VPC+GMIC % GAP CLOSED,REF FIRST_CUT_PASS % GAP CLOSED,REF+V FIRST_CUT_PASS % GAP CLOSED,REF LAST_CUT_PASS % GAP CLOSED,REF+V LAST_CUT_PASS % GAP CLOSED
disj_terms,,,,,,,,,
0,0,5.825078,22.643643,15.814017,21.449193,58.184478,58.184478,58.184478,58.184478
2,0,5.825078,0.000000,0.000000,5.825078,58.184478,58.184478,58.184478,58.184478
4,0,5.825078,0.000000,0.000000,5.825078,58.184478,58.184478,58.184478,58.184478
8,16,5.825078,0.000000,0.000000,5.825078,58.184478,0.000000,58.184478,0.000000
16,2,5.825078,0.000000,0.000000,5.825078,58.184478,0.000000,58.184478,0.000000
32,66,5.825078,0.000000,0.000000,5.825078,58.184478,0.000000,58.184478,0.000000
64,66,5.825078,22.643643,15.814017,21.449193,58.184478,0.000000,58.184478,0.000000


G                      5.825078
DB                    22.643643
V                     15.814017
V+G                   21.449193
GurF                  58.184478
V+GurF                58.184478
GurL                  58.184478
V+GurL                58.184478
BEST VPC DISJ         64.000000
BEST GMIC+VPC DISJ    64.000000
BEST V+GurF DISJ       2.000000
BEST V+GurL DISJ       2.000000
Name: neos22_presolved, dtype: float64

### Table 1: average percent gap closed across different combinations of cuts

In [164]:
## TABLE 1: average percent gap closed across different combinations of cuts
## Create avg_df = average gap closed across instances
all_set_name = 'All'
good_vpc_set_name = tex_escape('≥10%')
avg_row_name = tex_escape('Avg (%)')
wins_row_name = 'Wins'

idx = pd.MultiIndex.from_product(
    [ [all_set_name, good_vpc_set_name], [avg_row_name, wins_row_name] ],
    names = ['Set', '']
)
    
ncols = len(best_df.columns)
nrows = len(idx)

col = best_df['V'].astype(float)
good_vpc_df = best_df[col >= 10.]

data = np.zeros((nrows, ncols), dtype=float)
data[0,:] = [best_df[col].mean() for col in best_df.columns]
data[2,:] = [good_vpc_df[col].mean() for col in best_df.columns]

# display(best_df.head())
avg_df = pd.DataFrame(
    data,
    columns = best_df.columns,
    index = idx,
    dtype = object
)

inst_col_name = '# inst'
avg_df[inst_col_name] = [len(best_df), 0, len(good_vpc_df), 0]

avg_df.iloc[1] = ["" for i in range(ncols+1)]
avg_df.iloc[3] = ["" for i in range(ncols+1)]

display(avg_df)

G         DB          V        V+G       GurF  \
Set                                                                         
All       Avg (\%)  14.049826  15.284429   9.693486  19.963506  23.804026   
          Wins                                                              
$\ge$10\% Avg (\%)  15.739101  35.884534  28.430837  33.945782  27.248941   
          Wins                                                              

                       V+GurF       GurL     V+GurL BEST VPC DISJ  \
Set                                                                 
All       Avg (\%)  28.578802  41.380841    44.9031     30.752239   
          Wins                                                      
$\ge$10\% Avg (\%)  39.846548  47.528289  58.026513     52.666667   
          Wins                                                      

                   BEST GMIC+VPC DISJ BEST V+GurF DISJ BEST V+GurL DISJ # inst  
Set                                                                             
All       Avg (\%)          25.128358        16.250746        18.304478    335  
          Wins                                                                  
$\ge$10\% Avg (\%)          48.868687        32.707071        31.535354     99  
          Wins

In [165]:
best_df['V+GurF'].mean()

## TABLE 2: gap closed by num leaves
## Note that ``best'' can be worse than for a single row
## because when no VPCs are generated, we assume the "no VPCs" results hold for Gurobi,
## but we do not count that for the ``best'' calculation,
## since otherwise there is potential bias, as sometimes Gurobi does better without VPCs
sizes = [0,64]

shortcols = [
        'V+GurF',
    ]

gap_by_size_df = pd.DataFrame(
    columns = shortcols,
    index = sizes + ['Best'],
    # index = [str(size) + " leaves" for size in sizes]+['Best'],
    dtype = float,
)
zero_row_name = 0

# `grouped_df` will collect gap closed across instances, grouped by num terms
grouped_df = selected_gap_df.groupby(level='disj_terms').mean()
ungrouped_df = best_df.mean()

# For each of the columns (in shortcols),
# save the average value for each size
# (this will put in the right place as the index is based on sizes for both)
for col in shortcols:
    orig_col = map_short_to_cols[col]
    #gap_by_size_df.loc[2]['DB'] = best_df[orig_col].mean()
    gap_by_size_df[col] = grouped_df[orig_col]

# Fill in the 'Best' row, since that is currently stored in `gap_by_size_df` in the "0" row
gap_by_size_df.loc['Best'] = gap_by_size_df.loc[zero_row_name]

stubs = ['GurF']
for stub in stubs:
    col = 'V+'+stub
    # orig_col = map_short_to_cols[stub]
    gap_by_size_df[col][0] = ungrouped_df[stub]

display(grouped_df['REF+V FIRST_CUT_PASS % GAP CLOSED'])
display(gap_by_size_df)

disj_terms
0     28.578802
2     25.091156
4     25.293201
8     25.400077
16    26.084631
32    26.878820
64    28.091832
Name: REF+V FIRST_CUT_PASS % GAP CLOSED, dtype: float64

,V+GurF
0,23.804026
64,28.091832
Best,28.578802


### `wins_df`: num wins

In [ ]:
## Create num wins df
# x wins over y for an instance if x > y + EPS
#shortcols = avg_df.columns[0:-1]
wins_df = pd.DataFrame(
    np.zeros((len(cols), len(cols)), dtype=int),
    columns = cols,
    index = cols,
    dtype = int,
)

from itertools import permutations
for (ind1, ind2) in permutations(range(len(cols)), 2):
    wins_df.at[cols[ind1],cols[ind2]] = int(sum(best_df[cols[ind1]] > best_df[cols[ind2]] + EPS))
    wins_df.at[cols[ind2],cols[ind1]] = int(sum(best_df[cols[ind2]] > best_df[cols[ind1]] + EPS))

# Sets we are considering
# all_set = 'Wins (All)'
# good_vpc_set = 'Wins (V ≥ 10%)'
all_set = (all_set_name,wins_row_name)
good_vpc_set = (good_vpc_set_name,wins_row_name)

# "G" are wins relative to "V"
shortrefcol = 'V'
#refcol = 'VPC % GAP CLOSED'
#refcol = map_short_to_cols[shortrefcol]
refcol = shortrefcol
shortdestcol = 'G'
#col = 'GMIC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# "DB", "V", "V+G": wins are relative to "G"
shortrefcol = 'G'
#refcol = 'GMIC % GAP CLOSED'
#refcol = map_short_to_cols[shortrefcol]
refcol = shortrefcol
shortdestcol = 'DB'
#col = 'BEST DISJ % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

shortdestcol = 'V'
#col = 'VPC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

shortdestcol = 'V+G'
#col = 'VPC+GMIC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
destcol = shortdestcol
avg_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# "V+GurF" are wins relative to "GurF"
shortrefcol = 'GurF'
refcol = shortrefcol
shortdestcol = 'V+GurF'
destcol = shortdestcol
#col = map_short_to_cols[shortcol]
avg_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# "V+GurL" are wins relative to "GurL"
shortrefcol = 'GurL'
refcol = shortrefcol
shortdestcol = 'V+GurL'
destcol = shortdestcol
wins_df.at[shortdestcol,shortrefcol] = int(sum(best_df[destcol] > best_df[refcol] + EPS))
wins_df.at[shortrefcol,shortdestcol] = int(sum(best_df[refcol] > best_df[destcol] + EPS))
avg_df.at[all_set,shortdestcol] = wins_df.at[shortdestcol,shortrefcol]
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > good_vpc_df[refcol] + EPS)

# Count number of instances that have V+G > 0
shortdestcol = inst_col_name
#col = 'V+GurL'
destcol = 'V+G'
avg_df.at[all_set,shortdestcol] = sum(best_df[destcol] > EPS)
avg_df.at[good_vpc_set,shortdestcol] = sum(good_vpc_df[destcol] > EPS)

display(avg_df)
display(wins_df)

### Table 2: gap closed by num leaves

In [ ]:
## TABLE 2: gap closed by num leaves
## Note that ``best'' can be worse than for a single row
## because when no VPCs are generated, we assume the "no VPCs" results hold for Gurobi,
## but we do not count that for the ``best'' calculation,
## since otherwise there is potential bias, as sometimes Gurobi does better without VPCs
sizes = [0, 2, 4, 8, 16, 32, 64]

shortcols = [
        'DB',
        'V',
        'V+G',
        #'GurF',
        'V+GurF',
        #'GurL',
        'V+GurL',
    ]

gap_by_size_df = pd.DataFrame(
    columns = shortcols,
    index = sizes + ['Best'],
    # index = [str(size) + " leaves" for size in sizes]+['Best'],
    dtype = float,
)
zero_row_name = 0

# `grouped_df` will collect gap closed across instances, grouped by num terms
grouped_df = selected_gap_df.groupby(level='disj_terms').mean()
ungrouped_df = best_df.mean()

# For each of the columns (in shortcols),
# save the average value for each size
# (this will put in the right place as the index is based on sizes for both)
for col in shortcols:
    orig_col = map_short_to_cols[col]
    #gap_by_size_df.loc[2]['DB'] = best_df[orig_col].mean()
    gap_by_size_df[col] = grouped_df[orig_col]

# Fill in the 'Best' row, since that is currently stored in `gap_by_size_df` in the "0" row
gap_by_size_df.loc['Best'] = gap_by_size_df.loc[zero_row_name]

# Now update the zero row with correct values
col = 'DB'
gap_by_size_df[col][zero_row_name] = 0.

col = 'V'
gap_by_size_df[col][zero_row_name] = 0.

stubs = ['G', 'GurF', 'GurL']
for stub in stubs:
    col = 'V+'+stub
    # orig_col = map_short_to_cols[stub]
    gap_by_size_df[col][0] = ungrouped_df[stub]

# Reindex to add "leaves" to index
idx = [str(size) + " leaves" for size in sizes]+['Best']
reidx = {old_id : new_id for old_id, new_id in zip(gap_by_size_df.index,idx)}
gap_by_size_df.rename(reidx, inplace=True)

# display(grouped_df[gap_cols])
display(ungrouped_df)
display(gap_by_size_df)

### Export tables to LaTeX

#### Create helpful helper functions

In [ ]:
# Start with some helpful helper functions
def create_multirow_string(strval: str, num_rows: int = 2, alignment: str = 'c', extra_format: str = ""):
    """
    Wrap \p strval in a multirow environment for a table.
    """
    return \
        "{" + \
        "\\multirow[" + alignment + "]{"+ str(num_rows) + "}{*}{" + \
        (extra_format + "{" if extra_format != "" else "") + \
        str(strval) + \
        ("}" if extra_format != "" else "") + \
        "}" + "}"


def format_col_as_multirow(curr_series: pd.core.series.Series):
    start_val = ''
    start_row = -1
    end_row = -1
    for val in curr_series:
        end_row += 1
        is_last_row = end_row == len(curr_series)-1
        if val != start_val or is_last_row:
            num_rows = (end_row - start_row) + is_last_row
            if start_row >= 0 and num_rows > 1:
                multirow_string = create_multirow_string(str(start_val), num_rows = num_rows)
                curr_series[start_row] = multirow_string
                if is_last_row:
                    curr_series[end_row] = ""
            start_row = end_row
            start_val = val
        else:
            curr_series[end_row] = ""

# Some columns report both floats and ints
# This is a problem for siunitx that we need to fix explicitly
# We check for any int values in the table and apply a format to all of them
from math import floor, ceil

def is_val(val1: float, val2: float) -> bool:
    return abs(val1 - val2) < 1e-7

def is_int(val):
    """
    Checks whether given value should be treated as an int.

    Currently treats zero as a float always which is not ideal.
    """
    if isinstance(val, str) and val == '':
        return False
    try:
        floatval = float(val)
    except ValueError:
        # print("ValueError: ", val)
        return False
    # print("DEBUG:", val, ":", type(val))
    rounds_to_int = is_val(floatval, floor(floatval)) and is_val(floatval, ceil(floatval))
    is_zero = is_val(floatval, 0.0)
    # is_float_zero = (isinstance(val,str) and val.find('.') >= 0 and is_zero)
    return rounds_to_int and (not is_zero)
        
# def is_int_style(col : pd.core.series.Series):
#     # return ['background-color: green' if is_int(v) else '' for v in col]
#     return ['background-color: green' if is_int(v) else '' for v in col]

def int_format(val):
    if is_int(val):
        return "{\\tablenum[table-format=3]{" + str(val) + "}}"
    else:
        return val


# The styler from pandas has some limitations, particularly no way to add \midrule at arbitrary places
# display(TABLE1.style)
# display(styler)
def add_hline(latex: str, index_from_bottom: int) -> str:
    """
    Adds a horizontal `index` lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    lines.insert(len(lines) - index_from_bottom - 2, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')

# To add adjustbox, needs to be done after LaTeX string has been generated
def add_adjustbox_environment(latex: str) -> str:
    lines = latex.splitlines()
    start_env_ind = -1
    end_env_ind = -1
    curr_ind = -1
    for line in lines:
        curr_ind += 1
        if line.startswith(r"\begin{tabular}"):
            start_env_ind = curr_ind
        if line.startswith(r"\end{tabular}"):
            end_env_ind = curr_ind+1
    if (start_env_ind >= 0 and end_env_ind >= 0):
        lines.insert(start_env_ind, r'\begin{adjustbox}{width=1\textwidth}')
        lines.insert(end_env_ind+1, r'\end{adjustbox}')
    return '\n'.join(lines).replace('NaN', '')

def add_sisetup(latex: str) -> str:
    latex = \
"""
{
\sisetup{
    table-alignment-mode = format,
    table-number-alignment = center,
    table-format = 2.2,
}
""" + \
        latex + \
        "\n}"
    return latex

    

#### Format Table 1: gap closed and num wins

In [ ]:
# Format Table 1: gap closed and num wins

# Create copy of table then remove values we do not want (wins for 'G)
TABLE1 = avg_df.copy(deep=True)[[inst_col_name, 'G', 'DB', 'V', 'V+G', 'GurF', 'V+GurF', 'GurL', 'V+GurL']]

TABLE1['G'].loc[:,wins_row_name] = ""

# Process the column with # inst to only report number of instances for each set
TABLE1[inst_col_name].loc[:,wins_row_name] = ""
val = TABLE1[inst_col_name].loc[all_set_name,avg_row_name]
TABLE1[inst_col_name].loc[all_set_name,avg_row_name] = \
    create_multirow_string(str(val), extra_format=r"\tablenum[table-format=3]")
val = TABLE1[inst_col_name].loc[good_vpc_set_name,avg_row_name]
TABLE1[inst_col_name].loc[good_vpc_set_name,avg_row_name] = \
    create_multirow_string(str(val), extra_format=r"\tablenum[table-format=3]")

# Reset index to appear as cols
TABLE1.reset_index(inplace=True)

# Place column with # inst as second column
inst_col = TABLE1[inst_col_name]
TABLE1.drop(columns=[inst_col_name], inplace=True)
TABLE1.insert(loc=1, column=inst_col_name, value=inst_col)

# Set column should have multirow
setseries = TABLE1['Set']
format_col_as_multirow(setseries)

# for i in TABLE1.index:
#     curr_name = tex_escape(str(i))
#     print("Changing {} to {}".format(i, curr_name))
#     TABLE1.rename({i: curr_name}, inplace=True)
# print("")

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE1.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(col)
    TABLE1.rename({col: curr_col}, inplace=True, axis=1)

# Finally, apply the desired style
# styler.format({
#     ("Numeric", "Integers"): '\${}',
#     ("Numeric", "Floats"): '{:.3f}',
#     ("Non-Numeric", "Strings"): str.upper
# })
# styler.format_index(escape="latex", axis=0).format_index(escape="latex", axis=1)
# styler.hide(level=0,axis=0)
table1_str = TABLE1.style.\
    hide(axis=0).\
    format(formatter = int_format).\
    to_latex(
        #@{}l@{\hskip 5pt}
        column_format="""
        @{}l@{}
        S[table-format=2.0,table-auto-round,table-number-alignment=center]
        l
        *{1}{S[table-auto-round]}
        *{7}{S[table-auto-round]}
        @{}""",
        hrules = True,
        #clines = "skip-last;data",
        sparse_index = True,
        multirow_align = "c",
        # float_format="%.2f", 
        # escape=False, 
        siunitx=True,
        # index_names=False,
        #columns=['\# inst', 'G', 'DB', 'V', 'V+G', 'GurF', 'V+GurF', 'GurL', 'V+GurL']
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "tab:gap-closed-summary",
        caption = """
            Summary statistics for percent gap closed by VPCs.
            The wins row reports how many instances close at least $\epsilon$ more gap using DB, V, V+G compared to G on its own, V+GurF compared to GurF, and V+GurL compared to GurL.
        """,
        )

# Add a midrule between the two sets; the "3" is hand-coded but can be automated
table1_str = add_hline(table1_str, 3)

# Adjustbox environment sets width to pagewidth
table1_str = add_adjustbox_environment(table1_str)

# Set default siunitx options for this table
table1_str = add_sisetup(table1_str)

# display(TABLE1)
print(table1_str)

#### Format Table 2: percent gap closed by depth

In [ ]:
# Format Table 2: percent gap closed by depth
TABLE2 = gap_by_size_df.copy(deep=True)

# If we are not using the automatic tex-escaper, we need to do it ourselves
for col in TABLE2.columns:
    # curr_col = '{' + tex_escape(col) + '}'
    curr_col = tex_escape(col)
    TABLE2.rename({col: curr_col}, inplace=True, axis=1)


# Finally, apply the desired style
table2_str = TABLE2.style.\
    format(formatter = int_format).\
    to_latex(
        column_format="""
        @{}l
        *{5}{S[table-auto-round]}
        @{}""",
        hrules = True,
        sparse_index = True,
        multirow_align = "c",
        siunitx=True,
        convert_css = True,
        environment = "table",
        position_float = "centering",
        label = "tab:depth",
        caption = """
            Average percent gap closed broken down by the number of leaf nodes used to construct the partial branch-and-bound tree,
            for VPCs with and without GMICs, as well as at the root by \Gurobi{} after the first and last round of cuts. 
            ``0 leaves'' refers to the percent gap closed when no VPCs are used.
            ``Best'' refers to the maximum gap closed across all partial tree sizes.
        """,
        )

# Adjustbox environment sets width to pagewidth
# table2_str = add_adjustbox_environment(table2_str)

# Set default siunitx options for this table
table2_str = add_sisetup(table2_str)

# display(TABLE1)
print(table2_str)

# Section 2: Time tables

In [ ]:
## Create subset of dataframe relevant to time
time_df = df.loc[:, 
                [
                    'NUM DISJ TERMS',
                    'ROWS',
                    'COLS',
                    'LP OBJ',
                    'IP OBJ',
                    'FIRST REF OBJ',
                    'BEST REF OBJ',
                    'FIRST REF+V OBJ',
                    'FIRST REF BOUND',
                    'BEST REF BOUND',
                    'FIRST REF+V BOUND',
                    'FIRST REF ITERS',
                    'BEST REF ITERS',
                    'FIRST REF+V ITERS',
                    'FIRST REF NODES',
                    'BEST REF NODES',
                    'FIRST REF+V NODES',
                    'FIRST REF TIME',
                    'BEST REF TIME',
                    'AVG REF TIME',
                    'FIRST REF+V TIME',
                    'VPC_GEN_TIME',
                    'NUM VPC',
                    'NUM OBJ',
                    'ALL REF TIME',
                    'ExitReason']
               ]

newcol = 'REF+V W/CUTGEN TIME'
time_df[newcol] = time_df['FIRST REF+V TIME'] + time_df['VPC_GEN_TIME']

display(time_df.loc['bm23_presolved'])
#display(time_df.loc[("bm23_presolved",2)])

In [ ]:
## Select instances for time tables
#
# Criteria to filter gap closed instances:
# * ip opt val is known
# * max(nrows, ncols) ≤ 5K
# * Gur7 < 3600 (Gurobi is able to solve the instance to optimality within an hour either with or without using VPCs_
#
# 6 trees set
# * all six partial tree sizes produced VPCs

# Constants
MAX_ROWS = 5000
MAX_COLS = MAX_ROWS
MAX_TIME = 3600

# Information to save
selected_instances_dict = {} # dictionary of (original index, instance)
all6_instances_dict = {} # dictionary of (original index, instance)
skipped_instances_dict = {} # dictionary of (original index, instance)
error_instances_dict = {} # dictionary of (original index, instance)

num_attempts = np.zeros(len(instances), dtype=int)
num_timeouts = 0
num_errors = 0

i = 0
for inst in instances:
    skip_instance = False
    curr_df = df.loc[inst]
    
    # Count number of times instance appears
    num_attempts[i] = len(curr_df)

    if num_attempts[i] < 7:
        print("*** ERROR: Instance {:d} -- {}: {:d} < 7 attempts.".format(i, instances[i], num_attempts[i]))
        skip_instance = True
        num_errors += 1
        skipped_instances_dict[inst] = i
        error_instances_dict[inst] = i
    
    col = 'BEST REF TIME'
    mintime = float(curr_df.loc[0,col].min())

    if mintime > MAX_TIME - EPS:
        print("Skipping instance {:d} -- {}: best time taken is {:f} >= {:f}.".format(
                i, instances[i], mintime, MAX_TIME
            ))
        skip_instance = True
        skipped_instances_dict[inst] = i
        num_timeouts += 1

    # Check how many times VPCs were successfully generated
    num_successful_attempts = 0
    has_zero = False
    for curr_index, row in curr_df.iterrows():
        if row['NUM DISJ TERMS'] == 0:
            has_zero = True
            continue

        num_vpc = float(row['NUM VPC'])
        num_successful_attempts += (num_vpc > 0)

    if not has_zero:
        raise ValueError(
            "*** ERROR: Instance {:d} -- {}: has no bb0 entry.".format(
                i, instances[i], curr_index
            )
        )        
    
    if num_successful_attempts == 0:
        print("Skipping instance {:d} -- {}: no VPCs generated successfully for any number of terms.".format(i, instances[i], num_attempts[i]))
        skip_instance = True
        skipped_instances_dict[inst] = i
    else:        
        # Ensure IP objective value is known
        ip_obj = curr_df['IP OBJ'][curr_df.index[0]]
        if not isinstance(ip_obj,float):
            print(
                "Skipping instance {:d} -- {}: IP objective value ({}) is not detected to be a float value.".format(
                i, instances[i], ip_obj))
            skip_instance = True
            skipped_instances_dict[inst] = i
            
        # Ensure nrows and ncols is not too many
        nrows = curr_df['ROWS'][curr_df.index[0]]
        ncols = curr_df['COLS'][curr_df.index[0]]
        if (nrows > MAX_ROWS) or (ncols > MAX_COLS):
            print("Skipping instance {:d} -- {}: nrows = {:d} > {:d} or ncols = {:d} > {:d}.".format(
                    i, instances[i], nrows, ncols, MAX_ROWS, MAX_COLS))
            skip_instance = True
            skipped_instances_dict[inst] = i

    if not skip_instance:
        if num_successful_attempts == 6:
            all6_instances_dict[inst] = i
        #selected_instances_dict[len(selected_instances_dict)] = inst
        selected_instances_dict[inst] = i
        
    i = i + 1

num_selected_instances = len(selected_instances_dict)
num_all6_instances = len(all6_instances_dict)
print("Total number of errors: {}".format(num_errors))
print("Total number of timeouts: {}".format(num_timeouts))
print("Total number of selected instances: {}".format(num_selected_instances))
print("Total number of \"6 trees\" instances: {}".format(num_all6_instances))

selected_time_df = time_df.loc[selected_instances_dict.keys()]
selected_time_df.head(21)

In [ ]:
## DEBUG (check which instances were selected but do not have all six runs)

for key in selected_instances_dict.keys():
    if key not in all6_instances_dict.keys():
        print(key)

In [ ]:
## Fill in 0-row with min values across all rows
col_num_vpcs = 'NUM VPC'

map_cols_to_short_time = {
    'FIRST REF TIME'    : 'Gur1',
    'BEST REF TIME'     : 'Gur7',
    'FIRST REF+V TIME'  : 'V',
    newcol              : 'Total',
}

map_cols_to_short_nodes = {
    'FIRST REF NODES'   : 'Gur1',
    'BEST REF NODES'    : 'Gur7',
    'FIRST REF+V NODES' : 'V',
}

map_short_to_cols_time = {v: k for k, v in map_cols_to_short_time.items()}
map_short_to_cols_nodes = {v: k for k, v in map_cols_to_short_nodes.items()}

sizes = [2, 4, 8, 16, 32, 64]

time_cols = list(map_short_to_cols_time.keys())
node_cols = list(map_short_to_cols_nodes.keys())
display(time_cols, node_cols)

time_cols_long = [map_short_to_cols_time[col] for col in time_cols[2:]]
node_cols_long = [map_short_to_cols_nodes[col] for col in node_cols[2:]]

for inst in selected_time_df.index.get_level_values(0).unique():
    curr_df = selected_time_df.loc[inst].copy() # copy needed to not throw SettingWithCopyWarning
    curr_df_with_vpcs = curr_df[curr_df[col_num_vpcs] > 0]

    # Set 0-row to have max values across all rows for this instance
    best_vals = curr_df_with_vpcs[time_cols_long].min()
    selected_time_df.loc[(inst,0),time_cols_long] = best_vals

    best_vals = curr_df_with_vpcs[node_cols_long].min()
    selected_time_df.loc[(inst,0),node_cols_long] = best_vals

    # for ind in curr_df.index:
    #     if ind == 0:
    #         continue

    #     # Propogate GurF and GurL down
    #     subinds = [4,6]
    #     sel_gap = [gap_cols[i] for i in subinds]
    #     selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,sel_gap]

    #     # If no VPCs produced, the values for V+GurF and V+GurL have not been provided
    #     # We replace these by GurF and GurL
    #     # Currently disabled: update max for that column too (if disabled, we instead keep max as the value among those that generated VPCs)
    #     num_vpc = curr_df.loc[ind,col_num_vpcs]
    #     if num_vpc == 0:
    #         # print("Zero cuts for inst {} at depth {:d}".format(inst, ind))
    #         subinds = [5,7]
    #         refinds = [4,6]
    #         sel_gap = [gap_cols[i] for i in subinds]
    #         selected_gap_df.loc[(inst,ind),sel_gap] = curr_df.loc[0,[gap_cols[i] for i in refinds]].to_numpy()

    #         # for i in refinds:
    #         #     if curr_df.loc[0,gap_cols[i]] > selected_gap_df.loc[(inst,0),gap_cols[i+1]]:
    #         #         if curr_df.loc[0,gap_cols[i]] > 0:
    #         #             # print("DEBUG: Updating {} for inst {} from {:f} to {:f}".format(
    #         #             #     gap_cols[i+1], 
    #         #             #     inst, 
    #         #             #     selected_gap_df.loc[(inst,0),gap_cols[i+1]], 
    #         #             #     curr_df.loc[0,gap_cols[i]]))
    #         #         selected_gap_df.loc[(inst,0),gap_cols[i+1]] = curr_df.loc[0,gap_cols[i]]

newcol1 = 'MIN BB TIME'
selected_time_df[newcol1] = selected_time_df[['FIRST REF TIME','FIRST REF+V TIME']].min(axis=1)

newcol2 = 'MIN BB W/CUTGEN TIME'
selected_time_df[newcol2] = selected_time_df[['FIRST REF TIME','REF+V W/CUTGEN TIME']].min(axis=1)

newcol3 = 'MIN BB NODES'
selected_time_df[newcol3] = selected_time_df[['FIRST REF NODES','FIRST REF+V NODES']].min(axis=1)

display(selected_time_df.head(35).loc[:,[col_num_vpcs]+time_cols_long+node_cols_long+[newcol1,newcol2,newcol3]])


newshortcol1 = 'V7'
newshortcol2 = 'Total7'
newshortcol3 = 'V7'
map_cols_to_short_time[newcol1] = newshortcol1
map_cols_to_short_time[newcol2] = newshortcol2
map_cols_to_short_nodes[newcol3] = newshortcol3

map_short_to_cols_time[newshortcol1] = newcol1
map_short_to_cols_time[newshortcol2] = newcol2
map_short_to_cols_nodes[newshortcol3] = newcol3

time_cols.append(newshortcol1)
time_cols.append(newshortcol2)
node_cols.append(newshortcol3)

In [ ]:
## Prepare variables for row/col names

bb_classes = ['All', '6 wins']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000]
bucket_max = [3600, 3600, 3600, 3600]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]
# bucket_names = [classes[i] + ' [' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for i in range(num_classes) for j in range(num_buckets)]
# display(bucket_names)

bb_metrics = ['Gmean', 'Wins1', 'Wins7']

In [ ]:
## Prepare bb_df

cols = pd.MultiIndex.from_arrays([['Time']*len(time_cols) + ['Nodes']*len(node_cols), time_cols + node_cols], names = ['criterion', 'type'])
#bb_row_names = pd.MultiIndex.from_product([bb_buckets, bb_row_names], names=['bucket', 'metric'])
bb_row_names = pd.MultiIndex.from_product([bb_classes, bb_buckets, bb_metrics], names=['class', 'bucket', 'metric'])

bb_df = pd.DataFrame(
    columns = cols,
    index = bb_row_names,
    dtype = float
)

display(bb_df.loc[:,cols.get_level_values(0)=='Nodes'].head(6))
#display(bb_df.loc[(bb_classes[0], bb_buckets[1], bb_metrics[0]),:])
display(bb_df.loc[(bb_classes[0], bb_buckets, bb_metrics[0]),:])

In [ ]:
# ## DEBUG
# from statistics import geometric_mean
# tmp = [54, 24, 36]
# tmp = np.array(tmp)
# shift = 0

# def geo_mean(iterable):
#     a = np.array(iterable)
#     return a.prod()**(1.0/len(a))
# def geo_mean_overflow(iterable):
#     return np.exp(np.log(iterable).mean())

# display(geometric_mean(tmp+shift)-shift)
# display(geo_mean(tmp+shift)-shift)
# display(geo_mean_overflow(tmp+shift)-shift)

In [ ]:
## Create gmean_df
#   = shifted geometric mean of time taken across instances, in various buckets
#     and geomean of nodes too

# Custom functions for prior to python 3.8
# def geo_mean(iterable):
#     a = np.array(iterable)
#     return a.prod()**(1.0/len(a))
# def geo_mean_overflow(iterable):
#     return np.exp(np.log(iterable).mean())
from statistics import geometric_mean
SHIFT_TIME  = 60
SHIFT_NODES = 1000

num_inst = np.zeros(len(bb_df))
row_ind = 0

#bb_df.loc[(bb_classes[0], bb_buckets, bb_metrics[0]),:] = \
shortcols_time = time_cols
cols_time = [map_short_to_cols_time[shortcol] for shortcol in shortcols_time]
shortcols_nodes = node_cols
cols_nodes = [map_short_to_cols_nodes[shortcol] for shortcol in shortcols_nodes]

cols = cols_time + cols_nodes
shortcols = shortcols_time + shortcols_nodes

# First calculate stats for "all" instances
curr_df = selected_time_df.loc[:,cols]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0]

for i in range(num_buckets):
    curr_df = curr_df[curr_df['FIRST REF TIME'] > bucket_min[i]]
    bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[0]),('Time',shortcols_time)] = [geometric_mean(curr_df[col] + SHIFT_TIME) - SHIFT_TIME for col in cols_time]
    bb_df.loc[(bb_classes[0], bb_buckets[i], bb_metrics[0]),('Nodes',shortcols_nodes)] = [geometric_mean(curr_df[col] + SHIFT_NODES) - SHIFT_NODES for col in cols_nodes]
    print("row {:d}: {:d}".format(row_ind,len(curr_df)))
    num_inst[row_ind] = len(curr_df)
    row_ind += len(bb_metrics)

# Now calculate stats for "6 trees" instances
curr_df = selected_time_df.loc[all6_instances_dict.keys(),cols]
curr_df = curr_df[curr_df.index.get_level_values(1) == 0]

for i in range(num_buckets):
    curr_df = curr_df[curr_df['FIRST REF TIME'] > bucket_min[i]]
    bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[0]),('Time',shortcols_time)] = [geometric_mean(curr_df[col] + SHIFT_TIME) - SHIFT_TIME for col in cols_time]
    bb_df.loc[(bb_classes[1], bb_buckets[i], bb_metrics[0]),('Nodes',shortcols_nodes)] = [geometric_mean(curr_df[col] + SHIFT_NODES) - SHIFT_NODES for col in cols_nodes]
    print("row {:d}: {:d}".format(row_ind,len(curr_df)))
    num_inst[row_ind] = len(curr_df)
    row_ind += len(bb_metrics)

bb_df['NUM INST'] = num_inst

display(bb_df.loc[(bb_classes, bb_buckets, bb_metrics[0]),:])

In [ ]:
### DEBUG wins for bb
bb_time_wins_df = pd.DataFrame(
    np.zeros((len(cols), len(cols)), dtype=int),
    columns = cols,
    index = cols,
    dtype = int,
)

In [ ]:
## Create num wins df
# x wins over y for an instance if x > y + EPS
#shortcols = avg_df.columns[0:-1]
wins_df = pd.DataFrame(
    np.zeros((len(cols), len(cols)), dtype=int),
    columns = cols,
    index = cols,
    dtype = int,
)

from itertools import permutations
for (ind1, ind2) in permutations(range(len(cols)), 2):
    wins_df.at[cols[ind1],cols[ind2]] = int(sum(best_df[cols[ind1]] > best_df[cols[ind2]] + EPS))
    wins_df.at[cols[ind2],cols[ind1]] = int(sum(best_df[cols[ind2]] > best_df[cols[ind1]] + EPS))

# Sets we are considering
all_set = 'Wins (All)'
good_vpc_set = 'Wins (V ≥ 10%)'

# "G" are wins relative to "V"
shortrefcol = 'V'
#refcol = 'VPC % GAP CLOSED'
#refcol = map_short_to_cols[shortrefcol]
refcol = shortrefcol
shortcol = 'G'
#col = 'GMIC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
col = shortcol
avg_df.at[all_set,shortcol] = wins_df.at[shortcol,shortrefcol]
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > good_vpc_df[refcol] + EPS)

# "DB", "V", "V+G": wins are relative to "G"
shortrefcol = 'G'
#refcol = 'GMIC % GAP CLOSED'
#refcol = map_short_to_cols[shortrefcol]
refcol = shortrefcol
shortcol = 'DB'
#col = 'BEST DISJ % GAP CLOSED'
#col = map_short_to_cols[shortcol]
col = shortcol
avg_df.at[all_set,shortcol] = wins_df.at[shortcol,shortrefcol]
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > good_vpc_df[refcol] + EPS)

shortcol = 'V'
#col = 'VPC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
col = shortcol
avg_df.at[all_set,shortcol] = wins_df.at[shortcol,shortrefcol]
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > good_vpc_df[refcol] + EPS)

shortcol = 'V+G'
#col = 'VPC+GMIC % GAP CLOSED'
#col = map_short_to_cols[shortcol]
col = shortcol
avg_df.at[all_set,shortcol] = wins_df.at[shortcol,shortrefcol]
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > good_vpc_df[refcol] + EPS)

# "V+GurF" are wins relative to "GurF"
shortrefcol = 'GurF'
refcol = shortrefcol
shortcol = 'V+GurF'
col = shortcol
#col = map_short_to_cols[shortcol]
avg_df.at[all_set,shortcol] = wins_df.at[shortcol,shortrefcol]
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > good_vpc_df[refcol] + EPS)

# "V+GurL" are wins relative to "GurL"
shortrefcol = 'GurL'
refcol = shortrefcol
shortcol = 'V+GurL'
col = shortcol
wins_df.at[shortcol,shortrefcol] = int(sum(best_df[col] > best_df[refcol] + EPS))
wins_df.at[shortrefcol,shortcol] = int(sum(best_df[refcol] > best_df[col] + EPS))
avg_df.at[all_set,shortcol] = wins_df.at[shortcol,shortrefcol]
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > good_vpc_df[refcol] + EPS)

# Count number of instances that have V+G > 0
shortcol = '# inst'
#col = 'V+GurL'
col = 'V+G'
avg_df.at[all_set,shortcol] = sum(best_df[col] > EPS)
avg_df.at[good_vpc_set,shortcol] = sum(good_vpc_df[col] > EPS)

display(avg_df)
display(wins_df)

# Old code

In [ ]:
def filter_instances(df_ipopt, df_vpcbb):
    tmpnames = dfs[0].index
    tmpnames = tmpnames.intersection(dfs[1].index)
    tmpnames = tmpnames.intersection(dfs[2].index)
    zero_instances = []
    for inst_name in tmpnames:
        gaps = []
        for i in range(len(dfs)):
            gaps.append(dfs[i]['Gap'][inst_name])
        if max(gaps) < 1e-7:
            zero_instances.append(inst_name)
    return zero_instances

In [ ]:
df_hybrid = pd.read_csv(RESULTS_DIR+"hybrid.csv",sep=',',index_col=False)
df_hybrid = df_hybrid.set_index(df_hybrid[df_hybrid.columns[0]])
df_dense = pd.read_csv(RESULTS_DIR+"dense.csv",sep=',',index_col=False)
df_dense = df_dense.set_index(df_dense[df_dense.columns[0]])
df_sparse = pd.read_csv(RESULTS_DIR+"sparse.csv",sep=',',index_col=False)
df_sparse = df_sparse.set_index(df_sparse[df_sparse.columns[0]])
display(df_hybrid.head())
df_dense['Gap']['spar100-050-1'], df_hybrid['Gap']['spar100-050-1']

In [ ]:
hybrid_stub = "verbose_logfile_shd0.25c1x112_"
dense_stub = "verbose_logfile_sdd1.0c1x112_"
sparse_stub = "verbose_logfile_ssd0.25c1x112_"
inst_id = "091" # spar125-025-1 (boxqp)
# inst_id = "344" # be150.8.2 (maxcut)
# inst_id = "171" # pm1s_100.1 (biq)
df_spar125_hybrid = pd.read_csv(RESULTS_DIR+"hybrid/"+"%s%s.csv"%(hybrid_stub,inst_id), sep=',', header=1)
df_spar125_dense = pd.read_csv(RESULTS_DIR+"dense/"+"%s%s.csv"%(dense_stub,inst_id), sep=',', header=1)
df_spar125_sparse = pd.read_csv(RESULTS_DIR+"sparse/"+"%s%s.csv"%(sparse_stub,inst_id), sep=',', header=1)
#df_spar125_dense = pd.read_csv("%sdense.csv"%(inst), sep=',', header=1)
#df_spar125_sparse = pd.read_csv("%ssparse.csv"%(inst), sep=',', header=1)
df_spar125_hybrid.head()

# Plot gap closed and Gurobi time

In [ ]:
should_show_title = False
hybrid_stub = "verbose_logfile_shd0.25c1x112_"
dense_stub = "verbose_logfile_sdd1.0c1x112_"
sparse_stub = "verbose_logfile_ssd0.25c1x112_"

instance_ids = ["091", "344", "171"]
instance_names = ["spar125-025-1", "be150.8.2", "pm1s_100.1"]
fam_names = ["boxQP", "biq", "maxcut"]

# instance_ids = [instance_ids[2]]
# instance_names = [instance_names[2]]
# fam_names = [fam_names[2]]

out_ext = '-1hour.pdf'

i = -1
ymax = [200, 500, 125]

for (inst_id, inst_name, fam) in zip(instance_ids, instance_names, fam_names):
    i = i + 1
    #fam = df_instances['set'][inst_name]
    fname = RESULTS_DIR+"hybrid/"+"%s%s.csv"%(hybrid_stub,inst_id)
    hybrid_instance = pd.read_csv(fname, sep=',', header=1)
    fname = RESULTS_DIR+"dense/"+"%s%s.csv"%(dense_stub,inst_id)
    dense_instance = pd.read_csv(fname, sep=',', header=1)
    fname = RESULTS_DIR+"sparse/"+"%s%s.csv"%(sparse_stub,inst_id)
    sparse_instance = pd.read_csv(fname, sep=',', header=1)

    yaxes = ['Gurobi time', 'Gap closed']
    ylabels = ['LP time (s)', 'Gap closed (\%)']
    fnames = ['%s-time%s'%(inst_name,out_ext), '%s-gap%s'%(inst_name,out_ext)]
    tex_inst_name = tex_escape(inst_name)
    titles = [
        'LP resolve time (%s)'%(tex_inst_name),
        'Percent gap closed with respect to SDP optimum (%s)'%(tex_inst_name)
    ]

    j = 0
    for (yaxis, ylabel, fname, title) in zip(yaxes, ylabels, fnames, titles):
        j = j + 1
        ax = plt.gca()
        hybrid_line = hybrid_instance.plot(x='Time',y=yaxis,kind='line', label="\\texttt{HYBRID}", marker='', markersize=1, ax=ax)
        dense_line = dense_instance.plot(x='Time',y=yaxis,kind='line', label="\\texttt{DENSE}", marker='|', markersize=5, ax=ax)
        sparse_line = sparse_instance.plot(x='Time',y=yaxis,kind='line', label="\\texttt{SPARSE}", marker='.', markersize=5, ax=ax)
        if should_show_title:
            plt.title(title)
        plt.xlabel('Time (s)')
        plt.ylabel(ylabel)
        plt.xlim(0,3600)
        if j == 1:
            plt.ylim(top=ymax[i])
        #plt.xlim(0,86400)

        fig1 = plt.gcf()

        plt.tight_layout()
        plt.show()
        plt.draw()
        fig1.savefig(fname, dpi=300)

# Scatter plot of gap closed (dense vs hybrid)

In [ ]:
#col = ['blue', 'orange', 'purple']
# Replaced above with color-blind-friendly palette
col = ['#377eb8', '#ff7f00', '#4daf4a',
'#f781bf', '#a65628', '#984ea3',
'#999999', '#e41a1c', '#dede00']
col = [#'#4daf4a', 
       #'#ff7f00',
       '#377eb8',
        #'#f781bf', 
 #'#984ea3',
'#999999',       '#a65628', '#e41a1c', '#dede00']
inst_families = ['boxqp', 'maxcut', 'biq']
def set_marker_size_density(density):
    return density * 30 + 10
def set_marker_size_n(num_vars):
    return num_vars

sparse_ind = 0
dense_ind = 1
hybrid_ind = 2
dfs = [new_dfs[sparse_ind], new_dfs[dense_ind], new_dfs[hybrid_ind]] # sparse, dense, hybrid

tmpnames = dfs[hybrid_ind].index.intersection(df_dense.index)
gap_h = [dfs[hybrid_ind]['Gap'][tmpnames[i]] for i in range(len(tmpnames))]
gap_d = [dfs[dense_ind]['Gap'][tmpnames[i]] for i in range(len(tmpnames))]
n = [dfs[hybrid_ind]['n'][tmpnames[i]] for i in range(len(tmpnames))]
density_Q0 = df_hybrid['density_Q0'][tmpnames]
inst_family = [df_instances['set'][tmpnames[i]] for i in range(len(tmpnames))]

# Set integer ticks
from matplotlib.ticker import MaxNLocator
ax = plt.figure().gca()
ax.xaxis.set_major_locator(MaxNLocator(5,integer=True))
ax.yaxis.set_major_locator(MaxNLocator(5,integer=True))

# Do scatter plot
minx = 100
maxx = 0
miny = 100
maxy = 0
inst_fam_labeled = [False for i in range(len(inst_families))]
for i in range(len(tmpnames)):
    gap_x = gap_d[i] if gap_d[i].size == 1 else gap_d[i][0]
    gap_y = gap_h[i] if gap_h[i].size == 1 else gap_h[i][0]
    if (max(gap_x, gap_y) < 1e-3):
        continue
    density = density_Q0[i] if density_Q0[i].size == 1 else density_Q0[i][0]
    curr_n = n[i] if n[i].size == 1 else n[i][0]
    inst_fam = inst_family[i]
    size = set_marker_size_density(density)
    fam_ind = inst_families.index(inst_fam)
    #size = set_marker_size_density(density)
    #if inst_fam_labeled[fam_ind] or density < .95:
    size = set_marker_size_n(curr_n)
    if inst_fam_labeled[fam_ind] or (size not in [100,101]):
        label = ""
    else:
        label = inst_fam
        inst_fam_labeled[fam_ind] = True
    plt.scatter(gap_x, gap_y, label=label, s=size, color=col[fam_ind])
    if (gap_x > maxx):
        maxx = gap_x
    if (gap_x < minx):
        minx = gap_x
    if (gap_y > maxy):
        maxy = gap_y
    if (gap_y < miny):
        miny = gap_y

minx = 0
maxx = 100
        
plt.plot(range(int(minx),int(maxx)),range(int(minx),int(maxx)),'--', color='black')

#plt.title('Gap closed with respect to QCQP optimum (\%)')
plt.xlabel('\\texttt{DENSE} (\%)')
plt.ylabel('\\texttt{HYBRID} -- dense then sparse cuts (\%)')
plt.legend()

# locs, labels = ax.xticks()
# ax.set_xticks(np.arange(minx, maxx, step=1))
# ax.set_xticklabels(np.arange(minx, maxx, step=10))

fig = plt.gcf()
plt.tight_layout()
plt.show()
plt.draw()
fname = "hybrid_vs_dense_all_fams.pdf"
fig.savefig(fname, dpi=300)

## Second plot for only instances closing a lot of gap
ax = plt.figure().gca()
# Do scatter plot
minx = 100
maxx = 0
miny = 100
maxy = 0
inst_fam_labeled = [False for i in range(len(inst_families))]
for i in range(len(tmpnames)):
    gap_x = gap_d[i] if gap_d[i].size == 1 else gap_d[i][0]
    gap_y = gap_h[i] if gap_h[i].size == 1 else gap_h[i][0]
#     if tmpnames[i] == 'spar250-025-1':
#         print("here (dense = %f, hybrid = %f" % (gap_x, gap_y))
    if (max(gap_x, gap_y) < 1e-3):
        continue
    if (min(gap_x,gap_y) < 70):
        continue
    density = density_Q0[i] if density_Q0[i].size == 1 else density_Q0[i][0]
    curr_n = n[i] if n[i].size == 1 else n[i][0]
    inst_fam = inst_family[i]
    fam_ind = inst_families.index(inst_fam)
    #size = set_marker_size_density(density)
    #if inst_fam_labeled[fam_ind] or density < .95:
    size = set_marker_size_n(curr_n)
    if inst_fam_labeled[fam_ind] or (size not in [100,101]):
        label = ""
    else:
        label = inst_fam
        inst_fam_labeled[fam_ind] = True
    plt.scatter(gap_x, gap_y, label=label, s=size, color=col[fam_ind])
    if (gap_x > maxx):
        maxx = gap_x
    if (gap_x < minx):
        minx = gap_x
    if (gap_y > maxy):
        maxy = gap_y
    if (gap_y < miny):
        miny = gap_y

minx = 70
maxx = 100
        
plt.plot(range(int(minx),int(maxx)),range(int(minx),int(maxx)),'--', color='black')

#plt.title('Gap closed with respect to QCQP optimum (\%)')
plt.xlabel('\\texttt{DENSE} (\%)')
plt.ylabel('\\texttt{HYBRID} -- dense then sparse cuts (\%)')
plt.legend()

fig = plt.gcf()
plt.tight_layout()
plt.show()
plt.draw()
fname = "hybrid_vs_dense_all_fams_tail.pdf"
fig.savefig(fname, dpi=300)

# Extract first hour from each instance using verbose logfiles

In [ ]:
import os, fnmatch

def parse_verbose_batch(fname):
    is_batch = fname.endswith("_001.csv") or fname.endswith("_002.csv") or fname.endswith("_003.csv") or fname.endswith("_004.csv")
    breaklines = [0]
    inst_names = []
    with open(fname) as f:
        if is_batch:
            lines = f.read().splitlines()
            for i in range(len(lines)):
                line = lines[i]
                if (i==0):
                    inst_names.append(line)
                if (line.split(',')[0] == 'DONE!') and (i+2 < len(lines)):
                    breaklines.append(i+2)
                    inst_names.append(lines[i+2])
        else:
            inst_names.append(f.readline().rstrip('\n'))
            
    return inst_names, breaklines


def get_df_row(inst_name, fname, skiprows, nrows, target_time):
    header_row = 1
    if nrows != None:
        nrows = nrows-header_row
    curr_df = pd.read_csv(fname, sep=',', header=header_row, skiprows=skiprows, nrows=nrows)
    #display(curr_df, skiprows, nrows) ### DEBUG
    last_time = 0
    found_index = -1
    for index,row in curr_df.iterrows():
        found_index = index
        curr_time = row['Time']
        curr_gur_time = row['Gurobi time']
        if float(curr_time) + float(curr_gur_time) >= target_time:
            break
    found_index = min(found_index, len(curr_df)-2)
    found_row = curr_df.iloc[found_index,:]
    found_row.name = inst_name
    return found_row
        

dense_stub = "verbose_logfile_sdd1.0c1x*_"
sparse_stub = "verbose_logfile_ssd0.25c1x*_"
hybrid_stub = "verbose_logfile_shd0.25c1x*_"
exp_types = ["sparse", "dense", "hybrid"]
stubs = [sparse_stub, dense_stub, hybrid_stub]
RESULTS_DIR = "../results/saved/"
target_time = 3600.
new_dfs = []

#for (exp,stub,new_df) in zip(exp_types,stubs,new_dfs):
for i in range(len(exp_types)):
#for i in range(1): ### DEBUG
    exp = exp_types[i]
    stub = stubs[i]
    new_df = pd.DataFrame(columns = df_spar125_hybrid.columns)
    files = os.listdir(RESULTS_DIR + '/' + exp)
    files = fnmatch.filter(files, stub + "*")
    files.sort()
    #files = [files[217]] ### DEBUG
    # Now reach each file and get the last gap closed and Gurobi time for a 1 hour timelimit
    for f_stub in files:
        fname = RESULTS_DIR + '/' + exp + '/' + f_stub
        inst_names, starts = parse_verbose_batch(fname)
        for j in range(len(inst_names)):
            inst_name = inst_names[j]
            if len(inst_name) == 0:
                continue
            if j < len(inst_names)-1:
                nrows = starts[j+1] - starts[j]-2
            else:
                nrows = None
            #display(inst_name, starts[j], nrows, target_time) ### DEBUG
            found_row = get_df_row(inst_name, fname, starts[j], nrows, target_time)
            #display(found_row) ### DEBUG
            new_df = new_df.append(found_row)
            #if inst_name == "bqp50-10":
                #display(exp, stub, new_df.loc[inst_name])
    new_df.insert(len(new_df.columns), 'n', df_instances['n'][new_df.index])
    new_df = new_df.rename(columns={"Gap closed": "Gap"})
    new_dfs.append(new_df)

In [ ]:
#inst = 'spar020-100-1'
inst = 'bqp50-9'
display(new_dfs[0].loc[inst])
display(new_dfs[1].loc[inst])
display(new_dfs[2].loc[inst])

In [ ]:
#col = ['blue', 'orange', 'purple']
# Replaced above with color-blind-friendly palette
col = [#'#4daf4a', 
       #'#ff7f00',
       '#377eb8',
        #'#f781bf', 
 #'#984ea3',
'#999999',       '#a65628', '#e41a1c', '#dede00']
inst_families = ['boxqp']
densities = [.33, .67, 1]
def set_marker_size_density(density):
    return density * 20 + 10
def set_marker_size_n(num_vars):
    return num_vars

sparse_ind = 0
dense_ind = 1
hybrid_ind = 2
dfs = [new_dfs[sparse_ind], new_dfs[dense_ind], new_dfs[hybrid_ind]] # sparse, dense, hybrid

tmpnames = dfs[hybrid_ind].index.intersection(df_dense.index)
gap_h = [dfs[hybrid_ind]['Gap'][tmpnames[i]] for i in range(len(tmpnames))]
gap_d = [dfs[dense_ind]['Gap'][tmpnames[i]] for i in range(len(tmpnames))]
n = [dfs[hybrid_ind]['n'][tmpnames[i]] for i in range(len(tmpnames))]
density_Q0 = df_hybrid['density_Q0'][tmpnames]
inst_family = [df_instances['set'][tmpnames[i]] for i in range(len(tmpnames))]

# Set integer ticks
from matplotlib.ticker import MaxNLocator
ax = plt.figure().gca()
ax.xaxis.set_major_locator(MaxNLocator(5,integer=True))
ax.yaxis.set_major_locator(MaxNLocator(5,integer=True))

minx = 70
maxx = 101
        
plt.plot(range(int(minx),int(maxx)),range(int(minx),int(maxx)),'--', color='black')

# Do scatter plot
minx = 100
maxx = 0
miny = 100
maxy = 0
dense_fam_labeled = [False for i in range(len(densities))]
for i in (range(len(tmpnames))):
    gap_x = gap_d[i] if gap_d[i].size == 1 else gap_d[i][0]
    gap_y = gap_h[i] if gap_h[i].size == 1 else gap_h[i][0]
    if (max(gap_x, gap_y) < 1e-3):
        continue
    density = density_Q0[i] if density_Q0[i].size == 1 else density_Q0[i][0]
    curr_n = n[i] if n[i].size == 1 else n[i][0]
    inst_fam = inst_family[i]
    #size = set_marker_size_density(density)
    size = set_marker_size_n(curr_n)
    if inst_fam not in inst_families:
        continue
#    fam_ind = inst_families.index(inst_fam)
    if density <= densities[0]:
        fam_ind = 0
    elif density <= densities[1]:
        fam_ind = 1
    else:
        fam_ind = 2
    if dense_fam_labeled[fam_ind] or size != 100:
        label = ""
    else:
        if fam_ind == 0:
            label = "low density ($\\le 33\%$)"
        elif fam_ind == 1:
            label = "medium density ($34-67\%$)"
        else:
            label = "high density ($\ge 68\%$)"
        dense_fam_labeled[fam_ind] = True
    plt.scatter(gap_x, gap_y, label=label, s=size, color=col[fam_ind])
    if tmpnames[i] in ['spar250-025-1', 'spar250-025-2', 'spar250-025-3']:
        #plt.annotate(tmpnames[i], (gap_x+2.5, gap_y), ha='center', va='center', size='xx-large')
        plt.annotate("\\texttt{%s}"%tmpnames[i], (gap_x+3, gap_y), ha='center', va='center', size='xx-large')
    if tmpnames[i] in ['spar250-075-1', 'spar250-075-3']:
        #plt.annotate(tmpnames[i], (gap_x+2.5, gap_y), ha='center', va='center', size='xx-large')
        plt.annotate("\\texttt{%s}"%tmpnames[i], (gap_x+3, gap_y), ha='center', va='center', size='xx-large')
    if tmpnames[i] in ['spar250-075-2',]:
        #plt.annotate(tmpnames[i], (gap_x+2.5, gap_y-0.5), ha='center', va='center', size='xx-large')
        plt.annotate("\\texttt{%s}"%tmpnames[i], (gap_x+3, gap_y-0.5), ha='center', va='center', size='xx-large')
    if tmpnames[i] in ['spar200-025-1', 'spar200-025-2', 'spar200-025-3']:
        #plt.annotate(tmpnames[i], (gap_x-2.5, gap_y), ha='center', va='center', size='xx-large')
        plt.annotate("\\texttt{%s}"%tmpnames[i], (gap_x-2.75, gap_y), ha='center', va='center', size='xx-large')
    if tmpnames[i] in ['spar125-025-1', 'spar125-025-2', 'spar125-025-3']:
        #plt.annotate(tmpnames[i], (gap_x-2.5, gap_y), ha='center', va='center', size='xx-large')
        plt.annotate("\\texttt{%s}"%tmpnames[i], (gap_x-2.75, gap_y), ha='center', va='center', size='xx-large')
    if (gap_x > maxx):
        maxx = gap_x
    if (gap_x < minx):
        minx = gap_x
    if (gap_y > maxy):
        maxy = gap_y
    if (gap_y < miny):
        miny = gap_y

minx = 70
maxx = 101
        
#plt.plot(range(int(minx),int(maxx)),range(int(minx),int(maxx)),'--', color='black')

#plt.title('Gap closed with respect to QCQP optimum (\%)')
plt.xlabel('\\texttt{DENSE} (\%)')
plt.ylabel('\\texttt{HYBRID} -- dense then sparse cuts (\%)')
plt.legend()

#locs, labels = ax.xticks()
ax.set_xticks(np.arange(minx, maxx, step=10))
ax.set_yticks(np.arange(minx, maxx, step=10))
#ax.set_xticklabels(np.arange(minx, maxx, step=10))

fig = plt.gcf()
plt.tight_layout()
plt.show()
plt.draw()
fname = "hybrid-vs-dense-boxqp.pdf"
fig.savefig(fname, dpi=300)

In [ ]:
dfs[sparse_ind].to_csv("sparse_3600.csv")
dfs[dense_ind].to_csv("dense_3600.csv")
dfs[hybrid_ind].to_csv("hybrid_3600.csv")

In [ ]:
inst_families = ['boxqp', 'biq', 'maxcut']
fam_name = {'boxqp': 'BoxQP', 'biq': 'Biq', 'maxcut': 'MaxCut'}
ranges = [
    [[20,30], [40,50], [60,90], [100,125], [200,250]],
    #[[20,50], [60,80], [90,125], [200,250]],
    [[20,90], [100,100], [120,150], [200,250]],
    [[60,60], [80,80], [100,100], [150,225]]
]

def find_zero_gap_instances(dfs):
#     return [ 'gka6c',
#              'gka3a',
#              'gka7c',
#              'gka8a',
#              'bqp50-8',
#              'bqp50-7',
#              'bqp50-5',
#              'gka1a',
#              'bqp50-6',
#              'bqp50-2',
#              'gka2a',
#              'bqp50-1',
#              'bqp50-3',
#              'bqp50-4',
#              'bqp50-9']
    tmpnames = dfs[0].index
    tmpnames = tmpnames.intersection(dfs[1].index)
    tmpnames = tmpnames.intersection(dfs[2].index)
    zero_instances = []
    for inst_name in tmpnames:
        gaps = []
        for i in range(len(dfs)):
            gaps.append(dfs[i]['Gap'][inst_name])
        if max(gaps) < 1e-7:
            zero_instances.append(inst_name)
    # Add some instances that we manually have detected should be there
    if "bqp50-9" not in zero_instances:
        zero_instances.append("bqp50-9")
    return zero_instances

def print_gap_and_time_table(ranges, fam, dfs, target_time):
    # Exclude zero-gap instances
    zero_gap_instances = find_zero_gap_instances(dfs)
    
    # Ensure only instances common to all sets are taken
    tmpnames = dfs[0].index
    tmpnames = tmpnames.intersection(dfs[1].index)
    tmpnames = tmpnames.intersection(dfs[2].index)
    common_names = tmpnames
    
    tab = []
    total_num_inst = 0
    for curr_range in ranges:
        curr_row = []
        num_inst = -1
        stats = []
        for curr_df in dfs:
            # instances from max cut are off by one due to constant term in objective encoded as C
            lower_range = (curr_df['n'] >= curr_range[0])
            upper_range = (curr_df['n'] <= curr_range[1] + (1 if fam == 'maxcut' else 0))
            in_fam = df_instances['set'][curr_df.index] == fam
            nonzero_inst = ~curr_df.index.isin(zero_gap_instances)
            common_inst = curr_df.index.isin(common_names)
            curr_df = curr_df[lower_range & upper_range & in_fam & nonzero_inst & common_inst]
            curr_num_inst = len(curr_df)
            if num_inst >= 0:
                assert(curr_num_inst == num_inst)
            else:
                num_inst = curr_num_inst
            stats.append([curr_df['Gap'].mean(), curr_df['Gurobi time'].mean()])
        total_num_inst += num_inst
        if curr_range[0] != curr_range[1]:
            curr_row.append("$n \in [%d,%d]$"%(curr_range[0], curr_range[1]))
        else:
            curr_row.append("$n = %d$"%(curr_range[0]))
        curr_row.append("%d"%(num_inst))
        curr_row.extend([stats[i][0] for i in range(3)])
        curr_row.extend([stats[i][1] for i in range(3)])
        tab.append(curr_row)

    caption = (r"Results on %d %s instances for \SPARSE, \DENSE, and \HYBRID." % (total_num_inst, fam_name[fam])
               + " Results are averages over instances grouped by size, under a time limit of %s." % (target_time))
    return matrix2latex(
        tab, 
        None,
        "table", "center", "tabular",
        headerRow=[
            ["","",r"Gap closed (%)",r"Gap closed (%)",r"Gap closed (%)", "Last LP time (s)", "Last LP time (s)", "Last LP time (s)"],
            [r"Instance group",r"#",r"\SPARSE",r"\DENSE",r"\HYBRID",r"\SPARSE",r"\DENSE",r"\HYBRID"]
        ],
        alignment=r"@{} lc *{3}{c} *{3}{c} @{}",
        label="table:%s"%fam,
        formatColumn=["%s","%d","%.2f","%.2f","%.2f","%.2f","%.2f","%.2f"],
        summaryrows = 0,
        midruleIndex = [],
        caption=caption,
        position="t"
    )


full_dfs = [df_sparse, df_dense, df_hybrid]

print("\n## family: %s" % inst_families[0])
print(print_gap_and_time_table(ranges[0], inst_families[0], full_dfs, "1 day"))
print(print_gap_and_time_table(ranges[0], inst_families[0], new_dfs, "1 hour"))

print("\n## family: %s" % inst_families[1])
print(print_gap_and_time_table(ranges[1], inst_families[1], full_dfs, "1 day"))
print(print_gap_and_time_table(ranges[1], inst_families[1], new_dfs, "1 hour"))

print("\n## family: %s" % inst_families[2])
print(print_gap_and_time_table(ranges[2], inst_families[2], full_dfs, "1 day"))
print(print_gap_and_time_table(ranges[2], inst_families[2], new_dfs, "1 hour"))

In [ ]:
inst = 'spar200-025-1'
curr_range = [90,125]
curr_df = new_dfs[0]
fam = 'boxqp'
lower_range = (curr_df['n'] >= curr_range[0])
upper_range = (curr_df['n'] <= curr_range[1] + (1 if fam == 'maxcut' else 0))
in_fam = df_instances['set'][curr_df.index] == fam
#nonzero_inst = ~curr_df.index.isin(zero_gap_instances)
#common_inst = curr_df.index.isin(common_names)
curr_df = curr_df[lower_range & upper_range & in_fam]
curr_df.to_csv("sparse90-125.csv")

In [ ]:
inst = 'spar200-025-1'
curr_range = [90,125]
curr_df = new_dfs[2]
fam = 'boxqp'
lower_range = (curr_df['n'] >= curr_range[0])
upper_range = (curr_df['n'] <= curr_range[1] + (1 if fam == 'maxcut' else 0))
in_fam = df_instances['set'][curr_df.index] == fam
#nonzero_inst = ~curr_df.index.isin(zero_gap_instances)
#common_inst = curr_df.index.isin(common_names)
curr_df = curr_df[lower_range & upper_range & in_fam]
curr_df.to_csv("hybrid90-125.csv")

In [ ]:
zero_gap_instances = find_zero_gap_instances(new_dfs)
curr_df = df_sparse
df1 = curr_df[(~curr_df.index.isin(zero_gap_instances)) & (curr_df['n'] >= 20) & (curr_df['n'] <= 90)
         & (df_instances['set'][curr_df.index] == 'biq')]
curr_df = new_dfs[0]
df2 = curr_df[(~curr_df.index.isin(zero_gap_instances)) & (curr_df['n'] >= 20) & (curr_df['n'] <= 90)
         & (df_instances['set'][curr_df.index] == 'biq
df1['Gap closed'][df1.index], df2['Gap closed'][df1.index]

In [ ]:
# def get_df_row(inst_name, fname, skiprows, nrows, target_time):
#     curr_df = pd.read_csv(fname, sep=',', header=1, skiprows=skiprows, nrows=nrows)
#     last_time = 0
#     found_index = -1
#     for index,row in curr_df.iterrows():
#         found_index = index
#         curr_time = row['Time']
#         curr_gur_time = row['Gurobi time']
#         if float(curr_time) + float(curr_gur_time) >= target_time:
#             break
#     found_index = min(found_index, len(curr_df)-2)
#     found_row = curr_df.iloc[found_index,:]
#     found_row.name = inst_name
#     return found_row

# new_df = pd.DataFrame(columns = df_spar125_hybrid.columns)
# fname = RESULTS_DIR + '/sparse/verbose_logfile_ssd0.25c1x112_289.csv'
# inst_names, starts = parse_verbose_batch(fname)
# for j in range(len(inst_names)):
#     inst_name = inst_names[j]
#     if len(inst_name) == 0:
#         continue
#     #display(inst_name)
#     if j < len(inst_names)-1:
#         nrows = starts[j+1] - starts[j]-2
#     else:
#         nrows = None
#     found_row = get_df_row(inst_name, fname, starts[j], nrows, 3600.)
#     new_df = new_df.append(found_row)

# new_df.head()

In [ ]:
# hybrid_stub = "verbose_logfile_shd0.25c1x112_"
# dense_stub = "verbose_logfile_sdd1.0c1x112_"
# sparse_stub = "verbose_logfile_ssd0.25c1x112_"
# inst_id = "091" # spar125-025-1 (boxqp)
# # inst_id = "344" # be150.8.2 (maxcut)
# # inst_id = "171" # pm1s_100.1 (biq)
# #inst = "boxQP"
# #inst = "maxcut"
# #inst = "biq"
# df_spar125_hybrid = pd.read_csv(RESULTS_DIR+"hybrid/"+"%s%s.csv"%(hybrid_stub,inst_id), sep=',', header=1)
# df_spar125_dense = pd.read_csv(RESULTS_DIR+"dense/"+"%s%s.csv"%(dense_stub,inst_id), sep=',', header=1)
# df_spar125_sparse = pd.read_csv(RESULTS_DIR+"sparse/"+"%s%s.csv"%(sparse_stub,inst_id), sep=',', header=1)

In [ ]:

# fname = RESULTS_DIR+"hybrid/"+"%s%s.csv"%(hybrid_stub,inst_id)
# with open(fname,newline='\n') as f:
#     inst_name = f.readline()
#     display(inst_name.rstrip('\n'))
    
# new_df = pd.DataFrame(columns = df_spar125_hybrid.columns)
# #new_df.insert(0, "Name", value=None)
# last_time = 0
# target_time = 3600
# found_index = -1
# for index,row in df_spar125_hybrid.iterrows():
#     found_index = index
#     curr_time = row['Time']
#     curr_gur_time = row['Gurobi time']
#     if curr_time + curr_gur_time >= target_time:
#         break
# found_row = df_spar125_hybrid.iloc[found_index,:]
# found_row.name = "Test instance"
# new_df = new_df.append(found_row)
# new_df

In [ ]:
# new_df = pd.DataFrame(columns = df_spar125_hybrid.columns)
# new_df.insert(0, "Name", value=None)
# display(new_df.iloc[0,:])
# new_df.index[0]

In [ ]:
# new_df.rename(index={45: "testname"}, inplace=True)
# new_df.index[0]

In [ ]:
# test_df = df_spar125_hybrid.iloc[found_index,:]

In [ ]:
# test_df = pd.read_csv(RESULTS_DIR+"hybrid/"+"%s%s.csv"%(hybrid_stub,inst_id), sep=',', header=1)

In [ ]:
# fname = RESULTS_DIR+"hybrid/"+"%s%s.csv"%(hybrid_stub,inst_id)
# with open(fname,newline='\n') as f:
#     inst_name = f.readline()
#     display(inst_name.rstrip('\n'))

In [ ]:
# df_instances['set']['pm1s_100.1']

In [ ]:
# #ranges = [[20,50], [60,80], [90,125], [200,250]]
# ranges = [[20,50], [60,80], [90,125]]
# inst_families = ['boxqp', 'maxcut', 'biq']

# fam = inst_families[1]
# curr_range = ranges[0]
# curr_df = df_sparse
# lower_range = (curr_df['n'] >= curr_range[0])
# upper_range = (curr_df['n'] <= curr_range[1])
# in_fam = df_instances['set'][curr_df.index] == fam
# curr_df = curr_df[lower_range & upper_range & in_fam]
# #print(curr_df['Gap'].mean())
# #print(len(curr_df))
# print(curr_df)

In [ ]:
print(print_gap_and_time_table(ranges[0], inst_families[0], new_dfs, "1 hour"))

In [ ]:
ranges[0]

In [ ]:
new_dfs[0].loc['spar020-100-2']

In [ ]:
import os, fnmatch

def parse_verbose_batch(fname):
    is_batch = fname.endswith("_001.csv") or fname.endswith("_002.csv") or fname.endswith("_003.csv") or fname.endswith("_004.csv")
    breaklines = [0]
    inst_names = []
    with open(fname) as f:
        if is_batch:
            lines = f.read().splitlines()
            for i in range(len(lines)):
                line = lines[i]
                if (i==0):
                    inst_names.append(line)
                if (line.split(',')[0] == 'DONE!') and (i+2 < len(lines)):
                    breaklines.append(i+2)
                    inst_names.append(lines[i+2])
        else:
            inst_names.append(f.readline().rstrip('\n'))
            
    return inst_names, breaklines


def get_df_row(inst_name, fname, skiprows, nrows, target_time):
    header_row = 1
    if nrows != None:
        nrows = nrows-header_row-1
    print(inst_name, skiprows, nrows, target_time)
    curr_df = pd.read_csv(fname, sep=',', header=header_row, skiprows=skiprows, nrows=nrows)
    print(curr_df)
    last_time = 0
    found_index = -1
    for index,row in curr_df.iterrows():
        found_index = index
        curr_time = row['Time']
        curr_gur_time = row['Gurobi time']
        if float(curr_time) + float(curr_gur_time) >= target_time:
            break
    print(found_index, len(curr_df))
    found_index = min(found_index, len(curr_df)-1)
    print(found_index)
    found_row = curr_df.iloc[found_index,:]
    found_row.name = inst_name
    print(found_row)
    return found_row
        

dense_stub = "verbose_logfile_sdd1.0c1x*_"
sparse_stub = "verbose_logfile_ssd0.25c1x*_"
hybrid_stub = "verbose_logfile_shd0.25c1x*_"
exp_types = ["hybrid"]
stubs = [hybrid_stub]
RESULTS_DIR = "../results/saved/"
target_time = 3600.
new_dfs = []

#for (exp,stub,new_df) in zip(exp_types,stubs,new_dfs):
for i in range(len(exp_types)):
    exp = exp_types[i]
    stub = stubs[i]
    new_df = pd.DataFrame(columns = df_spar125_hybrid.columns)
    files = os.listdir(RESULTS_DIR + '/' + exp)
    files = fnmatch.filter(files, stub + "*")
    files.sort()
    # Now reach each file and get the last gap closed and Gurobi time for a 1 hour timelimit
    for f_stub in files:
        fname = RESULTS_DIR + '/' + exp + '/' + f_stub
        inst_names, starts = parse_verbose_batch(fname)
        for j in range(len(inst_names)):
            inst_name = inst_names[j]
            if len(inst_name) == 0:
                continue
            #display(inst_name)
            if j < len(inst_names)-1:
                nrows = starts[j+1] - starts[j]-2
            else:
                nrows = None
            found_row = get_df_row(inst_name, fname, starts[j], nrows, target_time)
            print(inst_name, j, starts[j], nrows, target_time, found_row)
            new_df = new_df.append(found_row)
            #if inst_name == "bqp50-10":
                #display(exp, stub, new_df.loc[inst_name])
            break
        break
    new_df.insert(len(new_df.columns), 'n', df_instances['n'][new_df.index])
    new_df = new_df.rename(columns={"Gap closed": "Gap"})
    new_dfs.append(new_df)

In [ ]:
starts

In [ ]:
fname = '../results/saved//hybrid/verbose_logfile_shd0.25c1x112_288.csv'
#fname = '../results/saved//sparse/verbose_logfile_ssd0.25c1x112_289.csv'
pd.read_csv(fname, sep=',', header=1, skiprows=0, nrows=None)
#fname = '../results/saved/hybrid/verbose_test.csv'
#pd.read_csv(fname)

In [ ]:
nrows

In [ ]:
nrows == None

In [ ]:
inst_families = ['boxqp', 'biq', 'maxcut']
fam_name = {'boxqp': 'BoxQP', 'biq': 'Biq', 'maxcut': 'MaxCut'}
ranges = [
    [[20,30], [40,50], [60,80], [90,125], [200,250]],
    #[[20,50], [60,80], [90,125], [200,250]],
    [[20,90]],
    [[60,60], [80,80], [100,100], [150,225]]
]

def find_zero_gap_instances(dfs):
    tmpnames = dfs[0].index
    tmpnames = tmpnames.intersection(dfs[1].index)
    tmpnames = tmpnames.intersection(dfs[2].index)
    zero_instances = []
    for inst_name in tmpnames:
        gaps = []
        for i in range(len(dfs)):
            gaps.append(dfs[i]['Gap'][inst_name])
        if max(gaps) < 1e-7:
            zero_instances.append(inst_name)
    return zero_instances

def print_gap_and_time_table(ranges, fam, dfs, target_time):
    # Exclude zero-gap instances
    zero_gap_instances = find_zero_gap_instances(dfs)
    
    # Ensure only instances common to all sets are taken
    tmpnames = dfs[0].index
    tmpnames = tmpnames.intersection(dfs[1].index)
    tmpnames = tmpnames.intersection(dfs[2].index)
    common_names = tmpnames
    
    tab = []
    total_num_inst = 0
    for curr_range in ranges:
        curr_row = []
        num_inst = -1
        stats = []
        for curr_df in dfs:
            # instances from max cut are off by one due to constant term in objective encoded as C
            lower_range = (curr_df['n'] >= curr_range[0])
            upper_range = (curr_df['n'] <= curr_range[1] + (1 if fam == 'maxcut' else 0))
            in_fam = df_instances['set'][curr_df.index] == fam
            nonzero_inst = ~curr_df.index.isin(zero_gap_instances)
            common_inst = curr_df.index.isin(common_names)
            curr_df = curr_df[lower_range & upper_range & in_fam & nonzero_inst & common_inst]
            curr_num_inst = len(curr_df)
            if num_inst >= 0:
                assert(curr_num_inst == num_inst)
            else:
                num_inst = curr_num_inst
            stats.append([curr_df['Gap'].mean(), curr_df['Gurobi time'].mean()])
        total_num_inst += num_inst
        if curr_range[0] != curr_range[1]:
            curr_row.append("$n \in [%d,%d]$"%(curr_range[0], curr_range[1]))
        else:
            curr_row.append("$n = %d$"%(curr_range[0]))
        curr_row.append("%d"%(num_inst))
        curr_row.extend([stats[i][0] for i in range(3)])
        curr_row.extend([stats[i][1] for i in range(3)])
        tab.append(curr_row)

    caption = (r"Results on %d %s instances for \SPARSE, \DENSE, and \HYBRID." % (total_num_inst, fam_name[fam])
               + " Results are averages over instances grouped by size, under a time limit of %s." % (target_time))
    return matrix2latex(
        tab, 
        None,
        "table", "center", "tabular",
        headerRow=[
            ["","",r"Gap closed (%)",r"Gap closed (%)",r"Gap closed (%)", "Last LP time (s)", "Last LP time (s)", "Last LP time (s)"],
            [r"Instance group",r"#",r"\SPARSE",r"\DENSE",r"\HYBRID",r"\SPARSE",r"\DENSE",r"\HYBRID"]
        ],
        alignment=r"@{} lc *{3}{c} *{3}{c} @{}",
        label="table:%s"%fam,
        formatColumn=["%s","%d","%.2f","%.2f","%.2f","%.2f","%.2f","%.2f"],
        summaryrows = 0,
        midruleIndex = [],
        caption=caption,
        position="t"
    )

print(print_gap_and_time_table(ranges[1], inst_families[1], full_dfs, "1 day"))
print(print_gap_and_time_table(ranges[1], inst_families[1], new_dfs, "1 hour"))

In [ ]:
fam = 'biq'
dfs = new_dfs
tmpnames = dfs[0].index
tmpnames = tmpnames.intersection(dfs[1].index)
tmpnames = tmpnames.intersection(dfs[2].index)
common_names = tmpnames
#curr_df = dfs[1]
curr_df = dfs[1]
lower_range = (curr_df['n'] >= curr_range[0])
upper_range = (curr_df['n'] <= curr_range[1] + (1 if fam == 'maxcut' else 0))
sum(df_instances['set'][curr_df.index] == fam)
in_fam = df_instances['set'][curr_df.index] == fam
nonzero_inst = ~curr_df.index.isin(zero_gap_instances)
common_inst = curr_df.index.isin(common_names)
curr_df = curr_df[lower_range & upper_range & in_fam & nonzero_inst & common_inst]
# curr_num_inst = len(curr_df)

In [ ]:
#zero_gap_instances
#common_names
#common_inst
#in_fam
#curr_df = curr_df[lower_range & upper_range & in_fam & nonzero_inst & common_inst]
curr_df

In [ ]:
curr_df = dfs[1]

In [ ]:
lower_range = (curr_df['n'] >= curr_range[0])
upper_range = (curr_df['n'] <= curr_range[1] + (1 if fam == 'maxcut' else 0))
sum(df_instances['set'][curr_df.index] == fam)
nonzero_inst = ~curr_df.index.isin(zero_gap_instances)
common_inst = curr_df.index.isin(common_names)

In [ ]:
curr_df[lower_range & upper_range & in_fam]

In [ ]:
zero_gap_instances

In [ ]:
col = 'Gurobi time'
col = 'Gap'
#inst = 'spar125-025-1'
#inst = 'be150.8.2'
inst = 'pm1s_100.1'
display(df_sparse[col][inst])
display(df_dense[col][inst])
display(df_hybrid[col][inst])